# Otodom Da|taset

## Data Loading and Aggregation

In [46]:
import numpy as np
import pandas as pd

In [47]:
df_otodom = pd.read_csv('datasets/timestamps/2024-03-08-otodom_dataset_raw.csv')
df_2024_03_25_otodom_new_data = pd.read_csv('datasets/timestamps/2024-03-25-otodom-new_data.csv')
df_2024_03_25_otodom_updated_data = pd.read_csv('datasets/timestamps/2024-03-25-otodom-updated_data.csv')
df_2024_04_07_otodom_new_data = pd.read_csv('datasets/timestamps/2024-04-07-otodom-new_data.csv')
df_2024_04_07_otodom_updated_data = pd.read_csv('datasets/timestamps/2024-04-07-otodom-updated_data.csv')
df_2024_04_21_otodom_new_data = pd.read_csv('datasets/timestamps/2024-04-21-otodom-new_data.csv')
df_2024_04_21_otodom_updated_data = pd.read_csv('datasets/timestamps/2024-04-21-otodom-updated_data.csv')
df_2024_05_05_otodom_new_data = pd.read_csv('datasets/timestamps/2024-05-05-otodom-new_data.csv')
df_2024_05_05_otodom_updated_data = pd.read_csv('datasets/timestamps/2024-05-05-otodom-updated_data.csv')

In [48]:
def update_dataframe(oldest_df, updated_df, new_df):
    def set_url_as_index(df):
        if 'url' in df.columns:
            df.set_index('url', inplace=True)
        else:
            raise ValueError("URL column not found. Please check the column names.")
    try:
        set_url_as_index(oldest_df)
        set_url_as_index(updated_df)
        set_url_as_index(new_df)
    except ValueError as e:
        print(e)
        return None
    
    oldest_df.update(updated_df)
    merged_df = pd.concat([oldest_df, new_df], axis=0)
    merged_df.reset_index(inplace=True)

    return merged_df

In [49]:
df_otodom = update_dataframe(df_otodom, df_2024_03_25_otodom_updated_data, df_2024_03_25_otodom_new_data)
df_otodom = update_dataframe(df_otodom, df_2024_04_07_otodom_updated_data, df_2024_04_07_otodom_new_data)
df_otodom = update_dataframe(df_otodom, df_2024_04_21_otodom_updated_data, df_2024_04_21_otodom_new_data)
df_otodom = update_dataframe(df_otodom, df_2024_05_05_otodom_updated_data, df_2024_05_05_otodom_new_data)

In [50]:
df_otodom.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14172 entries, 0 to 14171
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   url                   14172 non-null  object
 1   name/title            14172 non-null  object
 2   address               14172 non-null  object
 3   price                 14172 non-null  object
 4   area                  14172 non-null  object
 5   price-per-area        13492 non-null  object
 6   floor/store           13943 non-null  object
 7   no of rooms           14171 non-null  object
 8   year of construction  12627 non-null  object
 9   parking space         8531 non-null   object
 10  market                12627 non-null  object
 11  form of ownership     10867 non-null  object
 12  condition             11335 non-null  object
 13  rent                  6636 non-null   object
 14  heating               10426 non-null  object
 15  advertiser type       12627 non-null

## Data Preparation

### Price

#### Removing Missing Data, Unification, and Conversion to Numeric Types

In [51]:
# Rows without price
no_price = df_otodom[~df_otodom['price'].str.contains('\d')]
no_price['price'].count()

687

In [52]:
# Removing rows with missing price (no_price)
df_otodom.drop(no_price.index, inplace=True)
df_otodom['price'].count()

13485

#### Prices Given in Different Currencies

In [53]:
# Rows in EUR
different_currency = df_otodom[~df_otodom['price'].str.contains('zł', na=False)].copy()
different_currency['price'].count()

11

In [54]:
different_currency['price-per-area'] = different_currency['price-per-area'].str.replace(' ', '').str.extract(r'(\d+\.\d+|\d+)').astype(float)
different_currency['area'] = different_currency['area'].str.replace(',', '.').str.extract(r'(\d+\.\d+|\d+)').astype(float)

In [55]:
df_otodom.loc[different_currency.index]

,url,name/title,address,price,area,price-per-area,floor/store,no of rooms,year of construction,parking space,market,form of ownership,condition,rent,heating,advertiser type,elevator,outdoor area,building type,building material
1265,https://www.otodom.pl/pl/oferta/penthouse-232m...,Penthouse 232m w zabytkowej kamienicy Stare Mi...,"Nowy Świat, Stare Miasto, Kraków, małopolskie",1 149 999 EUR,"232,1 m²",21 451 zł/m²,4/4,5,1906,NaN,wtórny,pełna własność,do zamieszkania,NaN,gazowe,biuro nieruchomości,nie,NaN,kamienica,cegła
2056,https://www.otodom.pl/pl/oferta/penthouse-w-po...,Penthouse w pobliżu Rynku Głównego,"marszałka JózefaPiłsudskiego, Nowy Świat, Star...",1 150 000 EUR,232 m²,21 460 zł/m²,4/5,9,1906,NaN,wtórny,pełna własność,do zamieszkania,1 700 zł,gazowe,biuro nieruchomości,tak,"balkon, taras",kamienica,brak informacji
3144,https://www.otodom.pl/pl/oferta/5-pokoi-taras-...,5 pokoi | Taras 77m2 | Komórka | Centrum | Mar...,"ul. marsz. Józefa Piłsudskiego, Nowy Świat, St...",1 149 999 EUR,"232,1 m²",21 451 zł/m²,4/4,5,1908,garaż/miejsce parkingowe,wtórny,pełna własność,do zamieszkania,850 zł,gazowe,biuro nieruchomości,tak,"taras, balkon",kamienica,cegła
3977,https://www.otodom.pl/pl/oferta/chorwacja-spli...,Chorwacja Split luksusowy apartament,"Krowodrza, Krowodrza, Kraków, małopolskie",585 280 EUR,"91,45 m²",27 708 zł/m²,3/4,4,2025,garaż/miejsce parkingowe,pierwotny,pełna własność,do zamieszkania,100 EUR,elektryczne,prywatny,tak,"balkon, taras",apartamentowiec,silikat
4722,https://www.otodom.pl/pl/oferta/penthouse-tara...,Penthouse tarasem na dachu w kamienicy z windą,"Marszałka Józefa Piłsudskiego, Nowy Świat, Sta...",1 150 000 EUR,"228,7 m²",21 770 zł/m²,4/5,5,1906,NaN,wtórny,pełna własność,do zamieszkania,NaN,NaN,biuro nieruchomości,tak,"balkon, taras",kamienica,brak informacji
4908,https://www.otodom.pl/pl/oferta/penthouse-w-od...,Penthouse w odrestaurowanej kamienicy - taras ...,"ul. marsz. Józefa Piłsudskiego, Piasek, Stare ...",1 150 000 EUR,228 m²,21 896 zł/m²,4/5,5.0,1906,garaż/miejsce parkingowe,wtórny,pełna własność,do zamieszkania,220 zł,gazowe,biuro nieruchomości,tak,"taras, balkon",kamienica,cegła
4995,https://www.otodom.pl/pl/oferta/penthouse-star...,Penthouse Stare Miasto II Poziomy. Taras 70m2,"Marszałka Józefa Piłsudskiego, Nowy Świat, Sta...",1 150 000 EUR,"232,1 m²",21 451 zł/m²,4/5,5,1906,NaN,wtórny,NaN,NaN,NaN,NaN,biuro nieruchomości,nie,NaN,brak informacji,brak informacji
9359,https://www.otodom.pl/pl/oferta/penthouse-w-od...,Penthouse w odrestaurowanej kamienicy - taras ...,"ul. marsz. Józefa Piłsudskiego, Piasek, Stare ...",1 150 000 EUR,228 m²,21 837 zł/m²,4/5,5,1906,garaż/miejsce parkingowe,wtórny,pełna własność,do zamieszkania,220 zł,gazowe,biuro nieruchomości,tak,"taras, balkon",kamienica,cegła
13044,https://www.otodom.pl/pl/oferta/mieszkanie-swi...,Mieszkanie świetna lokalizacja Kraków-Śródmieście,"ulica Grochowska 27 B / 1B, Osiedle Oficerskie...",255 000 EUR,"107,3 m²",10 289 zł/m²,2/2,3,1981,garaż/miejsce parkingowe,wtórny,pełna własność,do zamieszkania,NaN,gazowe,prywatny,nie,"balkon, ogródek",szeregowiec,cegła
13565,https://www.otodom.pl/pl/oferta/krakowska-kami...,Krakowska kamienica,"ul. Mazowiecka, Krowodrza, Krowodrza, Kraków, ...",380 000 EUR,84 m²,19 585 zł/m²,5/6,3,1936,NaN,wtórny,pełna własność,do zamieszkania,880 zł,miejskie,prywatny,tak,balkon,kamienica,cegła


In [56]:
# converting prices to numeric values
df_otodom['price'] = df_otodom['price'].str.replace(' ', '').str.replace(',', '.')
df_otodom['price'] = df_otodom['price'].str.extract('(\d+\.\d+|\d+)').astype(float)

In [57]:
# recalculating price based on area and price per square meter given in PLN
different_currency.loc[:, 'price'] = (different_currency['price-per-area'] * different_currency['area'])
df_otodom.loc[different_currency.index, 'price'] = different_currency['price'].astype(float)
df_otodom.loc[different_currency.index, 'price']

1265     4978777.1
2056     4978720.0
3144     4978777.1
3977     2533896.6
4722     4978799.0
4908     4992288.0
4995     4978777.1
9359     4978836.0
13044    1104009.7
13565    1645140.0
13596    2714565.0
Name: price, dtype: float64

In [58]:
# Removing captured row with an apartment from Croatia
df_otodom.drop(index=3977, inplace=True)

In [59]:
df_otodom['price'].count()

13484

### Conversion of Column Values to Numeric Types

#### Area

In [60]:
df_otodom['area'] = df_otodom['area'].str.replace(' m²', '')
df_otodom['area'] = df_otodom['area'].str.replace(' ', '')
df_otodom['area'] = df_otodom['area'].str.replace(',', '.').astype(float)

#### Price per Square Meter

In [61]:
# df_otodom[df_otodom['price-per-area'].str.contains(r'\d', na=False).shape[0] & ~df_otodom['price-per-area'].str.contains('zł', na=False)].shape[0]
df_otodom[(df_otodom['price-per-area'].str.contains(r'\d', na=False)) & (~df_otodom['price-per-area'].str.contains('zł', na=False))].shape[0]

0

In [62]:
df_otodom[df_otodom['price-per-area'].isnull()].shape[0]

0

In [63]:
df_otodom['price-per-area'] = df_otodom['price-per-area'].str.replace(' zł/m²', '').str.replace(' ', '').str.replace(',', '.')
df_otodom['price-per-area'] = df_otodom['price-per-area'].astype(float)

In [64]:
# removing attics, service premises, entire floors, and errors
df_otodom.drop(df_otodom[(df_otodom['area'] > 200) & (df_otodom['price-per-area'] < 11000)].index, inplace=True)

#### Rent

In [65]:
# Rows in another currency
df_otodom[df_otodom['rent'].str.contains(r'\d', na=False) & ~df_otodom['rent'].str.contains('zł', na=False)]

,url,name/title,address,price,area,price-per-area,floor/store,no of rooms,year of construction,parking space,market,form of ownership,condition,rent,heating,advertiser type,elevator,outdoor area,building type,building material
6335,https://www.otodom.pl/pl/oferta/2m-wysoki-stan...,2M! Wysoki Standard| Gotowe| Balkon | Widok!,"Wola Duchacka, Podgórze Duchackie, Kraków, mał...",620000.0,35.55,17440.0,7/11,2,2019,NaN,wtórny,pełna własność,do zamieszkania,420 GBP,miejskie,biuro nieruchomości,tak,balkon,blok,brak informacji


In [66]:
eur_rent_price = df_otodom[df_otodom['rent'].str.contains('EUR', na=False)]
gbp_rent_price = df_otodom[df_otodom['rent'].str.contains('GBP', na=False)]

In [67]:
# converting rent values to numeric type
df_otodom['rent'] = df_otodom['rent'].str.replace(' ', '').str.replace(',', '.').str.extract('(\d+\.\d+|\d+)').astype(float)

In [68]:
import yfinance as yf

eur = yf.Ticker("EURPLN=X")
gbp = yf.Ticker("GBPPLN=X")
euro_exchange_rate = eur.info['fiftyDayAverage']
gbp_exchange_rate = gbp.info['fiftyDayAverage']
print(euro_exchange_rate)
print(gbp_exchange_rate)

4.304149
5.029401


In [69]:
df_otodom.loc[eur_rent_price.index, 'rent'] *= euro_exchange_rate
df_otodom.loc[gbp_rent_price.index, 'rent'] *= gbp_exchange_rate

In [70]:
# Average after cutting outliers for possible completion
mean_rent = df_otodom['rent'].mean()
std_rent = df_otodom['rent'].std()

filtered_std_rent = df_otodom[(df_otodom['rent'] > (mean_rent - std_rent)) & 
                        (df_otodom['rent'] < (mean_rent + std_rent))]

new_mean_rent = filtered_std_rent['rent'].mean()
new_mean_rent

542.4292394659827

#### Number of Rooms

In [71]:
df_otodom['no of rooms'] = df_otodom['no of rooms'].str.replace('więcej niż 10', '11')
df_otodom['no of rooms'] = df_otodom['no of rooms'].str.replace(' ', '')
print(df_otodom[df_otodom['no of rooms'].isna()].shape[0])
df_otodom.dropna(subset=['no of rooms'], inplace=True)

333


In [72]:
df_otodom['no of rooms'] = df_otodom['no of rooms'].str.extract('(\d+)').astype(int)

#### Floor

In [73]:
# Splitting 'floor/store' column into two separate columns
df_otodom[['floor/store', 'no of floors/stores in the building']] = df_otodom['floor/store'].str.split('/', expand=True)

In [74]:
# Removing rows without floor information
print(df_otodom[df_otodom['floor/store'].isnull()].shape[0])
df_otodom = df_otodom.dropna(subset=['floor/store'])

224


In [75]:
# Converting "parter" values to 0 in the "floor" column
df_otodom['floor/store'] = df_otodom['floor/store'].replace('parter', 0)

In [76]:
# Converting "suterena" values to -1 in the "floor" column
df_otodom['floor/store'] = df_otodom['floor/store'].replace('suterena', -1)

In [77]:
# Converting "floor" column to numeric type, replacing non-numeric values with NaN
df_otodom['floor/store'] = pd.to_numeric(df_otodom['floor/store'], errors='coerce')

# Removing rows containing NaN values in the "floor" column
df_otodom = df_otodom[~df_otodom['floor/store'].isna()]

In [78]:
df_otodom['floor/store'] = df_otodom['floor/store'].astype(int)

#### Number of Floors

In [79]:
print(df_otodom[df_otodom['no of floors/stores in the building'].isnull()].shape[0])
df_otodom = df_otodom.dropna(subset=['no of floors/stores in the building'])

487


In [80]:
df_otodom['no of floors/stores in the building'] = df_otodom['no of floors/stores in the building'].astype(int)

In [81]:
# Reorder
columns = list(df_otodom.columns)
columns.remove('no of floors/stores in the building')
target_index = columns.index('floor/store')
columns.insert(target_index + 1, 'no of floors/stores in the building')
df_otodom = df_otodom[columns]

In [82]:
# According to Wikipedia, the tallest residential building in Krk has 19 floors
# https://pl.wikipedia.org/wiki/Lista_najwyższych_budynków_w_Krakowie
print(df_otodom[df_otodom['no of floors/stores in the building'] > 19].shape[0])
df_otodom.drop(df_otodom[df_otodom['no of floors/stores in the building'] > 19].index, inplace=True)

1


#### Year of Construction

In [83]:
df_otodom['year of construction'] = pd.to_numeric(df_otodom['year of construction'], errors='coerce')

missing_below_1200 = df_otodom[df_otodom['year of construction'] < 1250]
missing_above_2026 = df_otodom[df_otodom['year of construction'] > 2026]
pd.concat([missing_above_2026, missing_below_1200])

,url,name/title,address,price,area,price-per-area,floor/store,no of floors/stores in the building,no of rooms,year of construction,...,market,form of ownership,condition,rent,heating,advertiser type,elevator,outdoor area,building type,building material
187,https://www.otodom.pl/pl/oferta/4-pokoje-nowe-...,"4 pokoje, nowe, Promocja -175 tys. taniej","Stare Podgórze, Podgórze, Kraków, małopolskie",998358.0,62.10,16077.0,1,4,4,2924.0,...,pierwotny,pełna własność,do wykończenia,NaN,miejskie,biuro nieruchomości,tak,balkon,apartamentowiec,brak informacji
7888,https://www.otodom.pl/pl/oferta/mieszkanie-na-...,"Mieszkanie na sprzedaż, Kraków 40,2m2","ul. Janusza Supniewskiego, Osiedle Oficerskie,...",699000.0,40.20,17388.0,0,2,2,20100.0,...,wtórny,pełna własność,do zamieszkania,350.0,miejskie,prywatny,nie,NaN,blok,cegła
10812,https://www.otodom.pl/pl/oferta/widokowe-3-pok...,"Widokowe 3 pok, 2 loggie, garaż podziemny, Ban...","ul. Stefana Banacha, Górka Narodowa, Prądnik B...",759000.0,49.47,15343.0,6,8,3,2924.0,...,pierwotny,pełna własność,do wykończenia,1.0,miejskie,biuro nieruchomości,tak,NaN,blok,brak informacji
6685,https://www.otodom.pl/pl/oferta/nowe-3-pokoje-...,Nowe: 3 pokoje na Ruczaju. Garaż podziemny.,"Ruczaj, Dębniki, Kraków, małopolskie",790300.0,56.45,14000.0,3,3,3,1024.0,...,pierwotny,pełna własność,do wykończenia,250.0,miejskie,biuro nieruchomości,tak,balkon,blok,pustak
11292,https://www.otodom.pl/pl/oferta/3-pokoje-na-bi...,3 Pokoje na Bieńczycach! Wysoki Standard!,"os. Wysokie, Bieńczyce, Bieńczyce, Kraków, mał...",669000.0,45.00,14867.0,3,4,3,450.0,...,wtórny,pełna własność,do zamieszkania,450.0,miejskie,biuro nieruchomości,nie,NaN,blok,cegła
11328,https://www.otodom.pl/pl/oferta/mieszkanie-na-...,Mieszkanie na sprzedaż- Mogilska-76 m2 + garaż,"ul. Mogilska, Osiedle Oficerskie, Grzegórzki, ...",1080000.0,76.00,14211.0,8,10,3,87.0,...,wtórny,pełna własność,do zamieszkania,1200.0,miejskie,biuro nieruchomości,tak,balkon,blok,wielka płyta
12986,https://www.otodom.pl/pl/oferta/nowy-apartamen...,Nowy apartament z ogródkiem 90 m2. Bez Prowizji.,"ul. Pana Tadeusza, Zabłocie, Podgórze, Kraków,...",794151.0,39.15,20285.0,0,7,2,204.0,...,pierwotny,NaN,do wykończenia,NaN,miejskie,biuro nieruchomości,tak,"balkon, ogródek",brak informacji,brak informacji


In [84]:
# Correcting "z placa" (with plaza)
df_otodom['year of construction'] = df_otodom['year of construction'].replace(2924.0, 2024)
df_otodom['year of construction'] = df_otodom['year of construction'].replace(1024.0, 2024)
df_otodom['year of construction'] = df_otodom['year of construction'].replace(204.0, 2024)
df_otodom['year of construction'] = df_otodom['year of construction'].replace(20100.0, 2010)
df_otodom['year of construction'] = df_otodom['year of construction'].replace(87.0, 1987)
df_otodom.drop(df_otodom[df_otodom['year of construction'] == 450.0].index, inplace=True)

In [85]:
# df_otodom['year of construction'] = df_otodom['year of construction'].fillna(-1).astype(int)
df_otodom['year of construction'].value_counts(dropna=False).head()

year of construction
NaN       2371
2024.0    1933
2023.0    1637
2025.0     553
2022.0     381
Name: count, dtype: int64

In [86]:
# Average after cutting outliers for possible completion
mean_year = df_otodom['year of construction'].mean()
std_year = df_otodom['year of construction'].std()

filtered_df = df_otodom[(df_otodom['year of construction'] > (mean_year - std_year)) & 
                        (df_otodom['year of construction'] < (mean_year + std_year))]

new_mean_year = filtered_df['year of construction'].mean()
new_mean_year = int(new_mean_year)
new_mean_year

2012

#### Market

In [87]:
df_otodom.loc[(df_otodom['market'].isna()) & (df_otodom['year of construction'] > 2022), 'market'] = 'pierwotny'
df_otodom.loc[(df_otodom['market'].isna()) & (df_otodom['year of construction'] < 2023), 'market'] = 'wtórny'

In [88]:
print(df_otodom['market'].isna().sum())
df_otodom.dropna(subset=['market'], inplace=True)

931


In [89]:
df_otodom['market'].value_counts(dropna=False)

market
wtórny       7439
pierwotny    3950
Name: count, dtype: int64

#### Ownership Type

In [90]:
df_otodom['form of ownership'].value_counts(dropna=False)

form of ownership
pełna własność                       7983
NaN                                  2999
spółdzielcze wł. prawo do lokalu      293
udział                                111
użytkowanie wieczyste / dzierżawa       3
Name: count, dtype: int64

In [91]:
# According to the law in effect in 2007, it is not possible to establish cooperative ownership rights to a property
df_otodom.loc[(df_otodom['form of ownership'].isna()) & (df_otodom['year of construction'] > 2006), 'form of ownership'] = 'pełna własność'

#### Condition

In [92]:
df_otodom['condition'].value_counts(dropna=False)

condition
do zamieszkania    4887
do wykończenia     3444
NaN                2549
do remontu          509
Name: count, dtype: int64

#### Parking

In [93]:
df_otodom['parking space'].value_counts(dropna=False)

parking space
garaż/miejsce parkingowe    6354
NaN                         5035
Name: count, dtype: int64

In [94]:
df_otodom['parking space'] = df_otodom['parking space'].notna()
df_otodom['parking space'].value_counts(dropna=False)

parking space
True     6354
False    5035
Name: count, dtype: int64

#### Heating

In [95]:
df_otodom['heating'].value_counts(dropna=False)

heating
miejskie         7212
NaN              2123
gazowe           1295
inne              395
elektryczne       211
kotłownia         151
piece kaflowe       2
Name: count, dtype: int64

#### Balcony

In [96]:
df_otodom['outdoor area'].value_counts(dropna=False)

outdoor area
balkon                    6336
NaN                       2793
ogródek                    570
balkon, taras              496
balkon, ogródek, taras     368
taras                      290
ogródek, taras             281
balkon, ogródek            201
taras, balkon               34
taras, ogródek              17
taras, balkon, ogródek       3
Name: count, dtype: int64

In [97]:
df_otodom['outdoor area'] = df_otodom['outdoor area'].fillna('None')

def standardize_outdoor_area(area):
    if area == 'None':
        return area
    parts = sorted(set(area.replace(' ', '').split(',')))
    return ', '.join(parts)

df_otodom['outdoor area'] = df_otodom['outdoor area'].apply(standardize_outdoor_area)

#### Advertiser

In [98]:
df_otodom['advertiser type'].value_counts(dropna=False)

advertiser type
biuro nieruchomości    9928
prywatny               1305
deweloper               156
Name: count, dtype: int64

In [99]:
df_otodom.loc[(df_otodom['advertiser type'].isna()), 'advertiser type'] = 'biuro nieruchomości'

#### Elevator

In [100]:
df_otodom['elevator'].value_counts(dropna=False)

elevator
tak    6131
nie    5258
Name: count, dtype: int64

In [101]:
# According to the building law in effect since 1960
df_otodom.loc[df_otodom['elevator'].isna() & (df_otodom['no of floors/stores in the building'] > 4), 'elevator'] = 'tak'
df_otodom['elevator'] = df_otodom['elevator'].map({'tak': True, 'nie': False, np.nan: False})
df_otodom['elevator'] = df_otodom['elevator'].astype(bool)

#### Building Type

In [102]:
df_otodom['building type'].value_counts(dropna=False)

building type
blok                5777
apartamentowiec     2221
brak informacji     1928
kamienica           1145
dom wolnostojący     208
szeregowiec           97
plomba                10
loft                   3
Name: count, dtype: int64

#### Material

In [103]:
df_otodom['building type'].value_counts(dropna=False)

building type
blok                5777
apartamentowiec     2221
brak informacji     1928
kamienica           1145
dom wolnostojący     208
szeregowiec           97
plomba                10
loft                   3
Name: count, dtype: int64

## Addresses - Preliminary

In [104]:
# Remove addresses outside Krakow
print(df_otodom[~df_otodom['address'].str.contains("Kraków", na=False)].shape[0])
df_otodom = df_otodom[df_otodom['address'].str.contains("Kraków", na=False)]

136


In [105]:
df_otodom.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11253 entries, 0 to 14171
Data columns (total 21 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   url                                  11253 non-null  object 
 1   name/title                           11253 non-null  object 
 2   address                              11253 non-null  object 
 3   price                                11253 non-null  float64
 4   area                                 11253 non-null  float64
 5   price-per-area                       11253 non-null  float64
 6   floor/store                          11253 non-null  int32  
 7   no of floors/stores in the building  11253 non-null  int32  
 8   no of rooms                          11253 non-null  int32  
 9   year of construction                 9817 non-null   float64
 10  parking space                        11253 non-null  bool   
 11  market                           

In [106]:
df_otodom.isnull().sum()

url                                       0
name/title                                0
address                                   0
price                                     0
area                                      0
price-per-area                            0
floor/store                               0
no of floors/stores in the building       0
no of rooms                               0
year of construction                   1436
parking space                             0
market                                    0
form of ownership                      1113
condition                              2540
rent                                   5229
heating                                2095
advertiser type                           0
elevator                                  0
outdoor area                              0
building type                             0
building material                         0
dtype: int64

In [107]:
nan_row_count = df_otodom.isna().any(axis=1).sum()
df_otodom.shape[0] - nan_row_count

4131

# Real Estate Dataset nieruchomosci-online

### Data Loading and Aggregation

In [108]:
df_nieruchomosci = pd.read_csv('datasets/timestamps/2024-03-08-nieruchomosci-online_dataset_raw.csv')
df_2024_03_25_nieruchomosci_new_data = pd.read_csv('datasets/timestamps/2024-03-25-nieruchomosci-online-new_data.csv')
df_2024_03_25_nieruchomosci_updated_data = pd.read_csv('datasets/timestamps/2024-03-25-nieruchomosci-online-updated_data.csv')
df_2024_04_07_nieruchomosci_new_data = pd.read_csv('datasets/timestamps/2024-04-07-nieruchomosci-online-new_data.csv')
df_2024_04_07_nieruchomosci_updated_data = pd.read_csv('datasets/timestamps/2024-04-07-nieruchomosci-online-updated_data.csv')
df_2024_04_21_nieruchomosci_new_data = pd.read_csv('datasets/timestamps/2024-04-21-nieruchomosci-online-new_data.csv')
df_2024_04_21_nieruchomosci_updated_data = pd.read_csv('datasets/timestamps/2024-04-21-nieruchomosci-online-updated_data.csv')
df_2024_05_05_nieruchomosci_new_data = pd.read_csv('datasets/timestamps/2024-05-05-nieruchomosci-online_new_data.csv')
df_2024_05_05_nieruchomosci_updated_data = pd.read_csv('datasets/timestamps/2024-05-05-nieruchomosci-online_updated_data.csv')

In [109]:
df_nieruchomosci = update_dataframe(df_nieruchomosci, df_2024_03_25_nieruchomosci_updated_data, df_2024_03_25_nieruchomosci_new_data)
df_nieruchomosci = update_dataframe(df_nieruchomosci, df_2024_04_07_nieruchomosci_updated_data, df_2024_04_07_nieruchomosci_new_data)
df_nieruchomosci = update_dataframe(df_nieruchomosci, df_2024_04_21_nieruchomosci_updated_data, df_2024_04_21_nieruchomosci_new_data)
df_nieruchomosci = update_dataframe(df_nieruchomosci, df_2024_05_05_nieruchomosci_updated_data, df_2024_05_05_nieruchomosci_new_data)

In [110]:
df_nieruchomosci.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9659 entries, 0 to 9658
Data columns (total 13 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   url                                  9659 non-null   object 
 1   name/title                           9659 non-null   object 
 2   address                              9290 non-null   object 
 3   price                                9536 non-null   object 
 4   area                                 9659 non-null   object 
 5   price-per-area                       9536 non-null   object 
 6   floor/store                          9659 non-null   object 
 7   no of floors/stores in the building  9027 non-null   float64
 8   no of rooms                          9659 non-null   object 
 9   year of construction                 9659 non-null   object 
 10  parking space                        9659 non-null   object 
 11  market                        

### Price and Price per Square Meter
#### Removing Rows with Missing Data

In [111]:
print(df_nieruchomosci[df_nieruchomosci['price'].isnull()].shape[0])
df_nieruchomosci.drop(df_nieruchomosci[df_nieruchomosci['price'].isnull()].index, inplace=True)

123


In [112]:
# checking if there are any values in the price-per-area column that do not contain numeric data
df_nieruchomosci[~df_nieruchomosci['price-per-area'].str.contains(r'\d')].shape[0]

0

In [113]:
# checking if there are any values in the price column that do not contain numeric data
df_nieruchomosci[~df_nieruchomosci['price'].str.contains(r'\d')].shape[0]

0

### Conversion of Prices Given in Different Currencies

In [114]:
different_currency_price = pd.concat([df_nieruchomosci[~df_nieruchomosci['price'].str.contains('zł')].copy(), df_nieruchomosci[~df_nieruchomosci['price-per-area'].str.contains('zł')].copy()]).drop_duplicates()
different_currency_price

,url,name/title,address,price,area,price-per-area,floor/store,no of floors/stores in the building,no of rooms,year of construction,parking space,market,form of ownership
153,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Piłsudskiego","Piłsudskiego, Stare Miasto, Kraków, małopolskie",1 149 999 €,"232,10 m²","4 954,76 €/m²",4,4.0,5,1906,parking strzeżony w pobliżu,wtórny,NaN
1120,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Piłsudskiego","Piłsudskiego, Stare Miasto, Kraków, małopolskie",1 149 999 €,"232,10 m²","4 954,76 €/m²",4,4.0,5,1908,garaż,wtórny,NaN
1151,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Piłsudskiego","Piłsudskiego, Stare Miasto, Kraków, małopolskie",1 150 000 €,"232,10 m²","4 954,76 €/m²",4,5.0,5,1906,parking publiczny / na ulicy,wtórny,NaN
2096,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Piłsudskiego","Piłsudskiego, Stare Miasto, Kraków, małopolskie",1 150 000 €,228 m²,"5 043,86 €/m²",4,5.0,5,1906,parking publiczny / na ulicy,wtórny,własność
2241,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Piłsudskiego","Piłsudskiego, Nowy Świat, Kraków, małopolskie",1 150 000 €,"228,70 m²","5 028,42 €/m²",4,5.0,5,1906,-,wtórny,NaN
2562,https://krakow.nieruchomosci-online.pl/mieszka...,Apartament Kraków,"Kraków, małopolskie",138 000 €,65 m²,"2 123,08 €/m²",21,45.0,2,2023,-,wtórny,NaN
2927,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, Plac Juliusza Kossaka","Plac Juliusza Kossaka, Stare Miasto, Kraków, m...",700 000 €,152 m²,"4 605,26 €/m²",1,4.0,4,1930,-,wtórny,NaN
2991,https://krakow.nieruchomosci-online.pl/mieszka...,Mieszkanie Kraków,"marszałka JózefaPiłsudskiego, Wawel, Kraków, m...",1 150 000 €,232 m²,"4 956,90 €/m²",4,5.0,9,1906,-,wtórny,NaN
3364,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie Stare Miasto, ul. Jana","Jana, Stare Miasto, Kraków, małopolskie",1 000 000 €,"221,19 m²",4 521 €/m²,4,4.0,4,1850,-,wtórny,NaN
8480,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Mazowiecka","Mazowiecka, Krowodrza, Kraków, małopolskie",370 000 €,84 m²,"4 404,76 €/m²",5,6.0,3,1936,-,wtórny,NaN


In [115]:
# converting prices to numeric values
df_nieruchomosci['price'] = df_nieruchomosci['price'].str.replace(' ', '').str.replace(',', '.').str.extract('(\d+\.\d+|\d+)').astype(float)

df_nieruchomosci['price-per-area'] = df_nieruchomosci['price-per-area'].str.replace(' ', '').str.replace(',', '.').str.extract('(\d+\.\d+|\d+)').astype(float)

In [116]:
print(df_nieruchomosci.loc[different_currency_price.index, ['price', 'price-per-area']])

          price  price-per-area
153   1149999.0         4954.76
1120  1149999.0         4954.76
1151  1150000.0         4954.76
2096  1150000.0         5043.86
2241  1150000.0         5028.42
2562   138000.0         2123.08
2927   700000.0         4605.26
2991  1150000.0         4956.90
3364  1000000.0         4521.00
8480   370000.0         4404.76


In [117]:
df_nieruchomosci.loc[different_currency_price.index, 'price'] *= euro_exchange_rate
df_nieruchomosci.loc[different_currency_price.index, 'price-per-area'] *= euro_exchange_rate

In [118]:
df_nieruchomosci.loc[different_currency_price.index, ['price', 'price-per-area']]

,price,price-per-area
153,4.949767e+06,21326.025299
1120,4.949767e+06,21326.025299
1151,4.949771e+06,21326.025299
2096,4.949771e+06,21709.524975
2241,4.949771e+06,21643.068915
2562,5.939726e+05,9138.052659
2927,3.012904e+06,19821.725224
2991,4.949771e+06,21335.236178
3364,4.304149e+06,19459.057629
8480,1.592535e+06,18958.743349


### Removing Rows with Shifted Data in Columns

In [119]:
print(df_nieruchomosci['no of rooms'].str.contains('-', na=False).sum())
df_nieruchomosci.drop(df_nieruchomosci[df_nieruchomosci['no of rooms'].str.contains('-', na=False)].index, inplace=True)

18


In [120]:
print(df_nieruchomosci['no of rooms'].str.contains(r'\D', na=False).sum())
df_nieruchomosci.drop(df_nieruchomosci[df_nieruchomosci['no of rooms'].str.contains(r'\D', na=False)].index, inplace=True)

67


### Number of Rooms and Floors

#### Conversion of Column Values to Numeric Types

In [121]:
df_nieruchomosci['no of rooms'] = df_nieruchomosci['no of rooms'].astype(int)

In [122]:
df_nieruchomosci['area'] = df_nieruchomosci['area'].str.replace(' m²', '').str.replace(',', '.').astype(float)

In [123]:
df_nieruchomosci[df_nieruchomosci['floor/store'].str.contains('parter')]

,url,name/title,address,price,area,price-per-area,floor/store,no of floors/stores in the building,no of rooms,year of construction,parking space,market,form of ownership
4,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Pustynna","Pustynna, Kliny, Kraków, małopolskie",1200000.0,71.06,16887.14,parter,1.0,3,2013,garaż w bryle budynku,wtórny,"własność, księga wieczysta"
5,https://krakow.nieruchomosci-online.pl/mieszka...,Apartament Kraków,"Wola Justowska, Kraków, małopolskie",1198680.0,71.35,16800.00,parter,2.0,4,2023,garaż w bryle budynku,pierwotny (zobacz inne nowe mieszkania w Krako...,własność
9,https://krakow.nieruchomosci-online.pl/mieszka...,"Apartament, ul. Szablowskiego","Szablowskiego, Bronowice, Kraków, małopolskie",599000.0,40.00,14975.00,parter,3.0,2,2001,tak,wtórny,"własność, księga wieczysta"
12,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Ułanów","Ułanów, Prądnik Czerwony, Kraków, małopolskie",790000.0,58.00,13620.69,parter,10.0,3,1962,-,wtórny,"własność, księga wieczysta"
18,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Obozowa","Obozowa, Ruczaj, Kraków, małopolskie",689900.0,51.94,13282.63,parter,7.0,2,2012,przynależne na ulicy,wtórny,"własność, księga wieczysta"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9637,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Reja","Reja 9, Żabiniec, Kraków, małopolskie",1100000.0,73.00,15068.49,parter,6.0,3,2005,-,NaN,NaN
9640,https://krakow.nieruchomosci-online.pl/mieszka...,"Apartament, ul. Kobierzyńska","Kobierzyńska, Dębniki, Kraków, małopolskie",899000.0,48.46,18551.38,parter,6.0,2,2022,tak,wtórny,NaN
9644,https://krakow.nieruchomosci-online.pl/mieszka...,"Apartament, ul. Królowej Jadwigi","Królowej Jadwigi, Wola Justowska, Kraków, mało...",894000.0,51.97,17202.00,parter,1.0,3,2023,w garażu podziemnym,pierwotny (zobacz inne nowe mieszkania w Krako...,NaN
9646,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Ułanów","Ułanów, Prądnik Czerwony, Kraków, małopolskie",790000.0,58.00,13620.69,parter,10.0,3,1962,-,wtórny,NaN


In [124]:
# Converting "parter" values to 0 in the "floor/store" column
df_nieruchomosci['floor/store'] = df_nieruchomosci['floor/store'].replace('parter', '0')

In [125]:
# Converting "suterena" values to -1 in the "floor/store" column
df_nieruchomosci['floor/store'] = df_nieruchomosci['floor/store'].replace('suterena', '-1')

In [126]:
# removing rows that contain non-numeric data in the 'floor/store' column
print(df_nieruchomosci[~df_nieruchomosci['floor/store'].str.contains(r'\d', na=False)].shape[0])
df_nieruchomosci.drop(df_nieruchomosci[~df_nieruchomosci['floor/store'].str.contains(r'\d')].index, inplace=True)

108


In [127]:
# shifted data
print(df_nieruchomosci[df_nieruchomosci['floor/store'].str.contains(r'[a-zA-Z]')].shape[0])
df_nieruchomosci.drop(df_nieruchomosci[df_nieruchomosci['floor/store'].str.contains(r'[a-zA-Z]')].index, inplace=True)

147


In [128]:
df_nieruchomosci['floor/store'] = df_nieruchomosci['floor/store'].astype(int)

In [129]:
print(df_nieruchomosci[df_nieruchomosci['no of floors/stores in the building'].isna()].shape[0])
df_nieruchomosci.dropna(subset=['no of floors/stores in the building'], inplace=True)
df_nieruchomosci['no of floors/stores in the building'] = df_nieruchomosci['no of floors/stores in the building'].astype(int)

185


In [130]:
# According to Wikipedia, the tallest residential building in Krk has 19 floors
# https://pl.wikipedia.org/wiki/Lista_najwyższych_budynków_w_Krakowie
print(df_nieruchomosci[df_nieruchomosci['no of floors/stores in the building'] > 19].shape[0])
df_nieruchomosci.drop(df_nieruchomosci[df_nieruchomosci['no of floors/stores in the building'] > 19].index, inplace=True)

1


In [131]:
# removing attics, service premises, entire floors, and errors
df_nieruchomosci.drop(df_nieruchomosci[(df_nieruchomosci['area'] > 200) & (df_nieruchomosci['price-per-area'] < 11000)].index, inplace=True)

### Year of Construction

In [132]:
df_nieruchomosci['year of construction'] = pd.to_numeric(df_nieruchomosci['year of construction'], errors='coerce')

missing_below_1200 = df_nieruchomosci[df_nieruchomosci['year of construction'] < 1250]
missing_above_2026 = df_nieruchomosci[df_nieruchomosci['year of construction'] > 2026]
pd.concat([missing_above_2026, missing_below_1200])

,url,name/title,address,price,area,price-per-area,floor/store,no of floors/stores in the building,no of rooms,year of construction,parking space,market,form of ownership
7393,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Banacha","Banacha, Prądnik Biały, Kraków, małopolskie",759000.0,49.47,15342.63,6,8,3,2924.0,tak,pierwotny (zobacz inne nowe mieszkania w Krako...,własność
2100,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Magnolii","Magnolii, Ruczaj, Kraków, małopolskie",790300.0,56.45,14000.00,3,3,3,1024.0,garaż,pierwotny (zobacz inne nowe mieszkania w Krako...,NaN


In [133]:
# Correcting "z placa" (with plaza)
df_nieruchomosci['year of construction'] = df_nieruchomosci['year of construction'].replace(2924.0, 2024)
df_nieruchomosci['year of construction'] = df_nieruchomosci['year of construction'].replace(1024.0, 2024)

In [134]:
df_nieruchomosci['year of construction'].value_counts(dropna=False)

year of construction
2024.0    1557
2023.0    1511
NaN        787
2025.0     702
2022.0     399
          ... 
1878.0       1
1921.0       1
1946.0       1
1350.0       1
1845.0       1
Name: count, Length: 153, dtype: int64

### Parking

In [135]:
df_nieruchomosci['parking space'] = df_nieruchomosci['parking space'].replace('-', 'parking publiczny / na ulicy')

In [136]:
df_nieruchomosci['parking space'].unique()

array(['tak', 'w garażu podziemnym', 'parking publiczny / na ulicy',
       'garaż w bryle budynku', 'garaż', 'przynależne na ulicy',
       'przynależne na terenie ogrodzonym', 'możliwość wykupienia',
       'garaż wolnostojący', 'parking strzeżony w pobliżu',
       'wiata garażowa'], dtype=object)

In [137]:
df_nieruchomosci['parking space'].value_counts(dropna=False)

parking space
parking publiczny / na ulicy         3474
w garażu podziemnym                  1602
tak                                  1124
garaż                                 855
przynależne na ulicy                  657
garaż wolnostojący                    470
przynależne na terenie ogrodzonym     243
garaż w bryle budynku                 218
parking strzeżony w pobliżu           194
możliwość wykupienia                  138
wiata garażowa                         27
Name: count, dtype: int64

In [138]:
# Copying data to a new column (might be useful for some model) and converting to boolean for consistency with Otodom
df_nieruchomosci['parking space details'] = df_nieruchomosci['parking space']
false_values = ['parking publiczny / na ulicy', 'możliwość wykupienia', 'parking strzeżony w pobliżu']
df_nieruchomosci['parking space'] = ~df_nieruchomosci['parking space'].isin(false_values)

### Market

In [139]:
valid_values = ['wtórny', 'pierwotny (zobacz inne nowe mieszkania w Krakowie)']

shifted_rows = ~df_nieruchomosci['market'].isin(valid_values) & df_nieruchomosci['market'].notna()

df_nieruchomosci.drop(df_nieruchomosci[shifted_rows].index, inplace=True)

In [140]:
df_nieruchomosci.loc[(df_nieruchomosci['market'].isna()) & (df_nieruchomosci['year of construction'] > 2022), 'market'] = 'pierwotny'
df_nieruchomosci.loc[(df_nieruchomosci['market'].isna()) & (df_nieruchomosci['year of construction'] < 2023), 'market'] = 'wtórny'
print(df_nieruchomosci['market'].isna().sum())
df_nieruchomosci.dropna(subset=['market'], inplace=True)

37


In [141]:
df_nieruchomosci['market'] = df_nieruchomosci['market'].replace('pierwotny (zobacz inne nowe mieszkania w Krakowie)', 'pierwotny')

### Ownership Type

In [142]:
# According to the law in effect since 2007, it is not possible to establish cooperative ownership rights to a property
df_nieruchomosci.loc[(df_nieruchomosci['form of ownership'].isna()) & (df_nieruchomosci['year of construction'] > 2006), 'form of ownership'] = 'własność'

In [143]:
ownership_mapping = {
    'własność': 'pełna własność',
    'własność, księga wieczysta': 'pełna własność',
    'spółdzielcze własnościowe, księga wieczysta': 'spółdzielcze wł. prawo do lokalu',
    'spółdzielcze własnościowe': 'spółdzielcze wł. prawo do lokalu',
    'udział w KW': 'udział',
    'udział ze wskazaniem, KW na budynku': 'udział',
    'księga wieczysta': 'pełna własność',
    'inna': 'użytkowanie wieczyste / dzierżawa',
    'udział': 'udział',
    'inna, księga wieczysta': 'użytkowanie wieczyste / dzierżawa',
    'Udział ze wskazaniem': 'udział',
    'Udział': 'udział',
    'udział, księga wieczysta': 'udział',
    'udział ze wskazaniem': 'udział',
    'Chce dopłacić': np.nan,
    'tak': np.nan,
    'udziały, księga wieczysta': 'udział',
    'Oczekuje dopłaty': np.nan,
    'KW budynek, udział ze wskazaniem': 'udział',
    'Spółdzielcza wł. z KW': 'spółdzielcze wł. prawo do lokalu'
}

df_nieruchomosci['form of ownership'] = df_nieruchomosci['form of ownership'].replace(ownership_mapping)

In [144]:
df_nieruchomosci['form of ownership'].value_counts(dropna=False)

form of ownership
pełna własność                       7020
NaN                                  1661
spółdzielcze wł. prawo do lokalu      228
użytkowanie wieczyste / dzierżawa      34
udział                                 13
Name: count, dtype: int64

In [145]:
nan_row_count = df_nieruchomosci.isna().any(axis=1).sum()
df_nieruchomosci.shape[0] - nan_row_count

7124

In [146]:
df_nieruchomosci.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8956 entries, 0 to 9658
Data columns (total 14 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   url                                  8956 non-null   object 
 1   name/title                           8956 non-null   object 
 2   address                              8956 non-null   object 
 3   price                                8956 non-null   float64
 4   area                                 8956 non-null   float64
 5   price-per-area                       8956 non-null   float64
 6   floor/store                          8956 non-null   int32  
 7   no of floors/stores in the building  8956 non-null   int32  
 8   no of rooms                          8956 non-null   int32  
 9   year of construction                 8208 non-null   float64
 10  parking space                        8956 non-null   bool   
 11  market                             

In [147]:
df_nieruchomosci.describe()

,price,area,price-per-area,floor/store,no of floors/stores in the building,no of rooms,year of construction
count,8.956000e+03,8956.000000,8956.000000,8956.000000,8956.000000,8956.000000,8208.000000
mean,9.738496e+05,58.190536,16862.503587,2.383318,4.865342,2.636668,2002.958577
std,6.402502e+05,28.827063,4838.352868,2.177881,2.449572,1.257195,37.862947
min,1.490000e+05,11.700000,19.140000,-1.000000,1.000000,1.000000,1300.000000
25%,6.600000e+05,40.500000,14000.000000,1.000000,3.000000,2.000000,1997.000000
50%,8.150000e+05,52.000000,16176.470000,2.000000,4.000000,3.000000,2022.000000
75%,1.063000e+06,67.000000,18779.340000,4.000000,6.000000,3.000000,2024.000000
max,1.600000e+07,550.000000,152594.090000,15.000000,17.000000,44.000000,2026.000000


In [148]:
df_nieruchomosci.isnull().sum()

url                                       0
name/title                                0
address                                   0
price                                     0
area                                      0
price-per-area                            0
floor/store                               0
no of floors/stores in the building       0
no of rooms                               0
year of construction                    748
parking space                             0
market                                    0
form of ownership                      1661
parking space details                     0
dtype: int64

## Addresses - Preliminary

In [149]:
df_nieruchomosci[~df_nieruchomosci['address'].str.contains("Kraków", na=False)]

,url,name/title,address,price,area,price-per-area,floor/store,no of floors/stores in the building,no of rooms,year of construction,parking space,market,form of ownership,parking space details


# Data Engineering

## Data Unification

In [150]:
common_columns = df_otodom.columns.intersection(df_nieruchomosci.columns)
df_apartments = pd.concat([df_otodom[common_columns], df_nieruchomosci[common_columns]], ignore_index=True)
df_apartments

,url,name/title,address,price,area,price-per-area,floor/store,no of floors/stores in the building,no of rooms,year of construction,parking space,market,form of ownership
0,https://www.otodom.pl/pl/oferta/gotowe-2-pokoj...,Gotowe| 2 pokoje| blisko centrum| Bonarka,"Wola Duchacka, Podgórze Duchackie, Kraków, mał...",719000.0,55.98,12844.00,0,8,3,2022.0,False,pierwotny,pełna własność
1,https://www.otodom.pl/pl/oferta/4-pok-mieszkan...,4-pok.mieszkanie z Sauną - Wysoki Standard ! 2...,"Biskupa Albina Małysiaka, Kobierzyn, Dębniki, ...",1350000.0,87.00,15517.00,2,4,4,2021.0,True,wtórny,pełna własność
2,https://www.otodom.pl/pl/oferta/3-pokoje-w-rza...,3 pokoje w rządowym programie kredyt 2%,"Bieżanów, Bieżanów-Prokocim, Kraków, małopolskie",599000.0,47.10,12718.00,1,4,3,2023.0,True,pierwotny,pełna własność
3,https://www.otodom.pl/pl/oferta/mieszkanie-ide...,"Mieszkanie idealne na start, 3 - pokoje!","ul. Erazma Jerzmanowskiego, Prokocim, Bieżanów...",833500.0,66.69,12498.00,3,8,3,2024.0,True,pierwotny,pełna własność
4,https://www.otodom.pl/pl/oferta/ul-lasowka-3-p...,"ul. Lasówka, 3 pokoje, 65m2 + taras 20m2!","ul. Lasówka, Płaszów, Podgórze, Kraków, małopo...",949000.0,65.81,14420.00,7,7,3,2023.0,False,wtórny,pełna własność
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20204,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Kobierzyńska","Kobierzyńska, Mateczny, Kraków, małopolskie",659000.0,35.50,18563.38,4,4,2,2024.0,True,pierwotny,pełna własność
20205,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Retoryka","Retoryka, Stare Miasto, Kraków, małopolskie",1750000.0,95.00,18421.05,2,4,3,1938.0,False,wtórny,pełna własność
20206,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Bieżanowska","Bieżanowska, Bieżanów, Kraków, małopolskie",659000.0,50.00,13180.00,9,11,3,1975.0,True,wtórny,NaN
20207,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Lubostroń","Lubostroń, Ruczaj, Kraków, małopolskie",715000.0,41.00,17439.02,1,3,2,2008.0,False,wtórny,pełna własność


In [151]:
df_apartments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20209 entries, 0 to 20208
Data columns (total 13 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   url                                  20209 non-null  object 
 1   name/title                           20209 non-null  object 
 2   address                              20209 non-null  object 
 3   price                                20209 non-null  float64
 4   area                                 20209 non-null  float64
 5   price-per-area                       20209 non-null  float64
 6   floor/store                          20209 non-null  int32  
 7   no of floors/stores in the building  20209 non-null  int32  
 8   no of rooms                          20209 non-null  int32  
 9   year of construction                 18025 non-null  float64
 10  parking space                        20209 non-null  bool   
 11  market                      

## Duplicates

In [152]:
def check_similarity(group):
    if len(group) > 1:
        price_mean = group['price'].mean()
        area_mean = group['area'].mean()
        price_range = price_mean * 0.01
        area_range = area_mean * 0.01
        similar_price = group['price'].between(price_mean - price_range, price_mean + price_range)
        similar_area = group['area'].between(area_mean - area_range, area_mean + area_range)
        return (similar_price & similar_area).rename('is_duplicate')
    else:
        return pd.Series(False, index=group.index, name='is_duplicate')

def fill_from_group(group):
    group['non_null_count'] = group.notna().sum(axis=1)
    sorted_group = group.sort_values('non_null_count', ascending=False)
    sorted_group.drop('non_null_count', axis=1, inplace=True)
    most_complete_row = sorted_group.iloc[0]
    for _, row in sorted_group.iloc[1:].iterrows():
        most_complete_row = most_complete_row.combine_first(row)
    return most_complete_row

def remove_duplicates(df, group_cols=None):
    if group_cols is None:
        group_cols = ['address', 'floor/store', 'no of floors/stores in the building', 'no of rooms']
    
    df['is_duplicate'] = df.groupby(group_cols, group_keys=False).apply(check_similarity)
    
    duplicates = df[df['is_duplicate']]
    non_duplicates = df[~df['is_duplicate']]
    
    most_complete_duplicates = duplicates.groupby(group_cols).apply(fill_from_group).reset_index(drop=True)
    
    filtered_df = pd.concat([non_duplicates, most_complete_duplicates], ignore_index=True)
    
    sorted_filtered_df = filtered_df.sort_values(by=['address', 'price', 'area'])
    
    return sorted_filtered_df

In [ ]:
df_filtered_sorted_apartments = remove_duplicates(df_apartments)
df_filtered_sorted_apartments.drop('is_duplicate', axis=1, inplace=True)

## Addresses

In [154]:
import re

# Regex pattern adjusted to exclude capturing prefixes directly
pattern = r'(ul\.|Aleja|aleja|pl\.|al\.)\s*([^,\d]+[\d]*\b)'

def update_address(row):
    if pd.isna(row['name/title']):
        return row['address']

    match = re.search(pattern, row['name/title'])
    if match:
        street_name = match.group(2).strip()
        if pd.isna(row['address']):
            updated_address = street_name
        else:
            if street_name.lower() not in row['address'].lower():
                updated_address = f"{street_name}, {row['address']}"
            else:
                updated_address = row['address']
    else:
        updated_address = row['address']
    
    return updated_address

In [155]:
df_filtered_sorted_apartments['address'] = df_filtered_sorted_apartments.apply(update_address, axis=1)
df_filtered_sorted_apartments['address'] = df_filtered_sorted_apartments['address'].str.replace(r'^ul\.\s+', '', regex=True)
df_filtered_sorted_apartments['address'] = df_filtered_sorted_apartments['address'].str.replace(r',\s*ul\.\s[^,\d]+,\s', ', ', regex=True)
df_filtered_sorted_apartments

,url,name/title,address,price,area,price-per-area,floor/store,no of floors/stores in the building,no of rooms,year of construction,parking space,market,form of ownership
15814,https://krakow.nieruchomosci-online.pl/mieszka...,Mieszkanie Kraków,"103171, Mistrzejowice, Kraków, małopolskie",1087000.0,74.80,14532.09,6,7,3,2016.0,True,wtórny,pełna własność
15415,https://krakow.nieruchomosci-online.pl/mieszka...,Mieszkanie Kraków,"2 Pułku Lotniczego, Kraków, małopolskie",550000.0,32.00,17187.50,4,8,1,NaN,False,wtórny,NaN
15937,https://krakow.nieruchomosci-online.pl/mieszka...,Mieszkanie Kraków,"2 Pułku Lotniczego, Kraków, małopolskie",550000.0,32.30,17028.00,4,10,1,1985.0,False,wtórny,NaN
13941,https://krakow.nieruchomosci-online.pl/mieszka...,"Apartament, ul. 28 Lipca 1943","28 Lipca 1943, Wola Justowska, Kraków, małopol...",2290000.0,155.16,14758.96,0,2,5,2009.0,True,wtórny,pełna własność
10191,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. 28 Lipca 1943","28 Lipca 1943, Wola Justowska, Kraków, małopol...",2399000.0,139.00,17258.99,1,2,3,2022.0,False,wtórny,pełna własność
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15803,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Żółkiewskiego","Żółkiewskiego, Grzegórzki, Kraków, małopolskie",695000.0,36.97,18799.03,1,2,2,1900.0,True,wtórny,NaN
16362,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Żółkiewskiego","Żółkiewskiego, Grzegórzki, Kraków, małopolskie",695000.0,37.98,18299.10,0,2,1,1900.0,False,wtórny,NaN
10889,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Żółkiewskiego","Żółkiewskiego, Grzegórzki, Kraków, małopolskie",1698500.0,79.00,21500.00,2,4,4,2024.0,False,pierwotny,pełna własność
11077,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Żółkiewskiego","Żółkiewskiego, Grzegórzki, Kraków, małopolskie",2722500.0,121.00,22500.00,2,4,4,2024.0,False,pierwotny,pełna własność


## Geolocation

In [196]:
from geopy.geocoders import Nominatim
import time

geolocator = Nominatim(user_agent="AGH_Student_Data_Exploration_Project")


def geocode(address):
    timeout = 10
    attempts = 0
    while attempts < 3:
        try:
            location = geolocator.geocode(address, timeout=timeout)
            if location:
                print(f"{location.address}, {location.latitude}, {location.longitude}")
                return pd.Series([location.address, location.latitude, location.longitude])
            else:
                return pd.Series([np.NaN, np.NaN, np.NaN])
        except Exception as e:
            print(f"Retry {attempts + 1}: Error during geocoding for address '{address}' - {str(e)}")
            attempts += 1
            timeout += 5
            time.sleep(2)
    return pd.Series([np.NaN, np.NaN, np.NaN])

In [198]:
unique_addresses = set(df_filtered_sorted_apartments['address'].unique())
len(unique_addresses)

3829

In [199]:
address_list = list(unique_addresses)

location_df = pd.DataFrame({
    'address': address_list,
    'manipulated address': address_list
})

In [197]:
import os

def geocode_and_save(temp_df, csv_path="datasets/geocoded_addresses.csv"):
    if 'location' not in temp_df:
        temp_df['location'] = np.NaN
    if 'latitude' not in temp_df:
        temp_df['latitude'] = np.nan
    if 'longitude' not in temp_df:
        temp_df['longitude'] = np.nan

    if os.path.exists(csv_path):
        existing_df = pd.read_csv(csv_path)
        existing_df = existing_df.drop(columns='manipulated address', errors='ignore')

        columns_to_drop = ['location', 'latitude', 'longitude']
        temp_df = temp_df.drop(columns=columns_to_drop, errors='ignore')

        merged_df = pd.merge(temp_df, existing_df, on='address', how='left')

        for col in columns_to_drop:
            if col + '_existing' in merged_df.columns:
                merged_df[col] = merged_df[col + '_existing'].combine_first(merged_df[col])
                merged_df.drop(col + '_existing', axis=1, inplace=True)

        needs_geocoding = merged_df['location'].isna()
        print(f"Rows needing geocoding: {needs_geocoding.sum()}")

        if needs_geocoding.any():
            geocoded_data = merged_df.loc[needs_geocoding, 'manipulated address'].apply(geocode)
            merged_df.loc[needs_geocoding, ['location', 'latitude', 'longitude']] = geocoded_data.values
        
        temp_df = merged_df
    else:
        geocoded_data = temp_df['manipulated address'].apply(geocode)
        temp_df[['location', 'latitude', 'longitude']] = geocoded_data.values

    temp_df.drop_duplicates(subset='address', keep='last', inplace=True)
    temp_df.to_csv(csv_path, index=False)
    print(f"Data saved to {csv_path}")
    
    return temp_df

In [188]:
geocoded_file = 'datasets/geocoded_addresses.csv'
df_geocoded = pd.read_csv(geocoded_file)
df_unique = df_geocoded.drop_duplicates()
location_df 
df_unique.to_csv(geocoded_file, index=False)

In [206]:
df2 = pd.read_csv('datasets/geocoded_from_xlsx.csv')
merged_df = pd.merge(df_geocoded, df2, on='address', how='left', suffixes=('', '_from_df2'))
df_geocoded.update(merged_df.filter(like='_from_df2').rename(columns=lambda x: x.replace('_from_df2', '')))
columns_to_drop = [col for col in df_geocoded.columns if '_from_df2' in col]
df_geocoded.drop(columns=columns_to_drop, axis=1, inplace=True)
df_geocoded.to_csv(geocoded_file, index=False)

In [209]:
location_df = geocode_and_save(location_df)

Rows needing geocoding: 0
Data saved to datasets/geocoded_addresses.csv


In [208]:
location_df[location_df['location'].isna()] 

,address,manipulated address,location,latitude,longitude


In [195]:
location_df.loc[location_df['location'].isna(), 'manipulated address'] = location_df['manipulated address'].str.replace(r',.*?, Kraków, małopolskie', ', Kraków, małopolskie', regex=True)

In [120]:
location_df.loc[location_df['location'].isna(), 'manipulated address'] = location_df['manipulated address'].str.replace(r'^(ul\. |ul\.|os\. |U |Ok\. |abp\. |al\. |mrj\.|Św\. |Aleja gen\. |Św\. |prof\. |gen\. )', '', regex=True)

In [122]:
phrases_to_remove = [
    "okolice", "Wykończone inwestycyjne mieszkanie", "garsoniera wypoczynkowa",
    "⭐️ obok tramwaj", "EKOPARK", r"\| brak pcc \|", "wykończone bez PCC",
    "przy UP", "/bez pośredników", "- dwupoziomowe mieszkanie w centrum",
    " ENG", r"\(Os\.Piastow", " z ogródkiem", " - Bezpośrednio", " - stan deweloperski",
    r"\/ nowy apartament\/", " - Wykończone", " Tbs"
    ]

# Create a regex pattern string to match any of the phrases
phrases_pattern = r'|'.join([re.escape(phrase) for phrase in phrases_to_remove])

# Apply regex only where location is NaN
location_df.loc[location_df['location'].isna(), 'manipulated address'] = \
    location_df.loc[location_df['location'].isna(), 'manipulated address'].str.replace(phrases_pattern, '', regex=True)


In [124]:
location_df.loc[location_df['location'].isna(), 'manipulated address'] = location_df[location_df['location'].isna()]['manipulated address'].str.replace(r'^-\s*,', ',', regex=True)
location_df.loc[location_df['location'].isna(), 'manipulated address'] = location_df[location_df['location'].isna()]['manipulated address'].str.replace(r'^iej,', 'a,', regex=True)
location_df.loc[location_df['location'].isna(), 'manipulated address'] = location_df[location_df['location'].isna()]['manipulated address'].str.replace(r'^ej,', 'a,', regex=True)

In [126]:
existing_df = pd.read_csv("geocoded_addresses.csv")
condition = existing_df['location'] == "Kraków, województwo małopolskie, Polska"
existing_df.loc[condition, ['location', 'latitude', 'longitude']] = None
existing_df.to_csv("geocoded_addresses.csv", index=False)

FileNotFoundError: [Errno 2] No such file or directory: 'geocoded_addresses.csv'

In [127]:
location_df[location_df['location'] == "Kraków, województwo małopolskie, Polska"]

,address,manipulated address,location,latitude,longitude
68013,"64108, Kraków, małopolskie","64108, Kraków, małopolskie","Kraków, województwo małopolskie, Polska",50.061947,19.936856
68429,"Kraków, małopolskie","Kraków, małopolskie","Kraków, województwo małopolskie, Polska",50.046943,19.997153


In [202]:
# import openpyxl
na_location = location_df[location_df['location'].isna()]
na_location.to_excel("temp.xlsx", index=False)

In [205]:
geocode_and_save(pd.read_excel("temp.xlsx"), 'datasets/geocoded_from_xlsx.csv')

Rows needing geocoding: 1
Krzemieniecka, Barycz, Kosocice, Swoszowice, Kraków, województwo małopolskie, 30-699, Polska, 49.9835607, 19.9961567
Data saved to datasets/geocoded_from_xlsx.csv


,address,manipulated address,location,latitude,longitude
0,"Kuźnicy K - OD WŁAŚCICIELA, Górka Narodowa, Pr...","Kuźnicy Kołłątajowskiej, Górka Narodowa, Prądn...","Kuźnicy Kołłątajowskiej, Górka Narodowa, Prądn...",50.098114,19.959952
1,"Krzemienicka Prokocim z ogrodem, Kosocice, Swo...","Krzemieniecka, Kosocice, Swoszowice, Kraków, ...","Krzemieniecka, Barycz, Kosocice, Swoszowice, K...",49.983561,19.996157


In [168]:
na_location[na_location['location'].isna()]

,address,manipulated address,location,latitude,longitude
1153,"Kuźnicy K - OD WŁAŚCICIELA, Górka Narodowa, Pr...","Kuźnicy K - OD WŁAŚCICIELA, Górka Narodowa, Pr...",None,None,None
2894,"Krzemienicka Prokocim z ogrodem, Kosocice, Swo...","Krzemienicka Prokocim z ogrodem, Kosocice, Swo...",None,None,None


In [210]:
merged_df = pd.merge(
    df_filtered_sorted_apartments, 
    location_df[['address', 'location', 'latitude', 'longitude']], 
    on='address', 
    how='left'
)

needs_update = merged_df['location'].isna()
merged_df.loc[needs_update, 'location'] = merged_df.loc[needs_update, 'location']
merged_df.loc[needs_update, 'latitude'] = merged_df.loc[needs_update, 'latitude']
merged_df.loc[needs_update, 'longitude'] = merged_df.loc[needs_update, 'longitude']

df_filtered_sorted_apartments = merged_df


In [211]:
df_filtered_sorted_apartments = df_filtered_sorted_apartments[df_filtered_sorted_apartments['location'] != "Kraków, województwo małopolskie, Polska"]

## Calculating Distance from the City Center

In [213]:
# https://en.wikipedia.org/wiki/Haversine_formula
def haversine(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    
    r = 6371
    
    return c * r

In [214]:
# Center of the Cloth Hall
city_center_lat = 50.061853457016106
city_center_lon = 19.937306011671154

In [ ]:
df_filtered_sorted_apartments['distance'] = df_filtered_sorted_apartments.apply(
    lambda row: haversine(city_center_lat, city_center_lon, row['latitude'], row['longitude']),
    axis=1
)

In [ ]:
# Extracting districts
df_filtered_sorted_apartments['district'] = df_filtered_sorted_apartments['location'].str.extract(r'(?:^|, )([^,]+), Kraków')
df_filtered_sorted_apartments

### Verification of District Assignment Accuracy

In [ ]:
df_filtered_sorted_apartments['district'].unique()

In [ ]:
df_filtered_sorted_apartments[df_filtered_sorted_apartments['district'] == 'Rajsko']

In [ ]:
df_filtered_sorted_apartments[df_filtered_sorted_apartments['district'] == 'Henryka Niewodniczańskiego']

In [ ]:
df_filtered_sorted_apartments[df_filtered_sorted_apartments['district'] == 'Tytusa Chałubińskiego']

In [ ]:
df_filtered_sorted_apartments[df_filtered_sorted_apartments['district'] == 'Wawel']

In [ ]:
# Correcting incorrectly assigned districts
df_filtered_sorted_apartments.loc[df_filtered_sorted_apartments['district'] == 'Tytusa Chałubińskiego', 'district'] = 'Swoszowice'
df_filtered_sorted_apartments[df_filtered_sorted_apartments['district'] == 'Henryka Niewodniczańskiego'] = 'Swoszowice'
df_filtered_sorted_apartments[df_filtered_sorted_apartments['district'] == 'Rajsko'] = 'Swoszowice'
df_filtered_sorted_apartments[df_filtered_sorted_apartments['district'] == 'Wawel'] = 'Stare Miasto'

In [217]:
print(df_filtered_sorted_apartments[df_filtered_sorted_apartments['district'].isna()].shape[0])
df_filtered_sorted_apartments = df_filtered_sorted_apartments.dropna(subset=['district'])

7


In [223]:
temp_df = geocode('Zgodna, 32-031 Kraków, małopolskie')
temp_df

Zgodna, Knapówka, Libertów, gmina Mogilany, powiat krakowski, województwo małopolskie, 30-444, Polska, 49.9782828, 19.895051


0    Zgodna, Knapówka, Libertów, gmina Mogilany, po...
1                                            49.978283
2                                            19.895051
dtype: object

In [218]:
df_filtered_sorted_apartments.isnull().sum()

url                                       0
name/title                                0
address                                   0
price                                     0
area                                      0
price-per-area                            0
floor/store                               0
no of floors/stores in the building       0
no of rooms                               0
year of construction                   1891
parking space                             0
market                                    0
form of ownership                      2379
location                                  0
latitude                                  0
longitude                                 0
distance                                  0
district                                  0
dtype: int64

In [219]:
nan_row_count = df_filtered_sorted_apartments.isna().any(axis=1).sum()
df_filtered_sorted_apartments.shape[0] - nan_row_count

14588

## Filling NaNs and Exporting Unified Dataset

In [ ]:
df_filtered_sorted_apartments['year of construction'] = df_filtered_sorted_apartments['year of construction'].fillna(new_mean_year)
df_filtered_sorted_apartments['form of ownership'] = df_filtered_sorted_apartments['form of ownership'].fillna('pełna własność')
df_filtered_sorted_apartments.to_csv('datasets/apartments_collective.csv')

## Removing Duplicates, Geolocation, Filling NaNs, and Exporting nieruchomosci-online Dataset

In [ ]:
df_filtered_sorted_nieruchomosci = remove_duplicates(df_nieruchomosci)
df_filtered_sorted_nieruchomosci.drop('is_duplicate', axis=1, inplace=True)
df_filtered_sorted_nieruchomosci

In [228]:
merged_df = pd.merge(
    df_filtered_sorted_nieruchomosci,
    location_df_nieruchomosci[['address', 'location', 'latitude', 'longitude']],
    on='address',
    how='left',
    suffixes=('', '_from_location')
)


# Check and update the original columns conditionally
if 'location_from_location' in merged_df.columns:
    merged_df['location'] = merged_df['location_from_location'].combine_first(merged_df['location'])
    merged_df['latitude'] = merged_df['latitude_from_location'].combine_first(merged_df['latitude'])
    merged_df['longitude'] = merged_df['longitude_from_location'].combine_first(merged_df['longitude'])

    # Drop the _from_location columns if they exist
    cols_to_drop = ['location_from_location', 'latitude_from_location', 'longitude_from_location']
    cols_to_drop = [col for col in cols_to_drop if col in merged_df.columns]
    merged_df.drop(cols_to_drop, axis=1, inplace=True)
else:
    needs_update = merged_df['location'].isna()
    merged_df.loc[needs_update, 'location'] = merged_df.loc[needs_update, 'location']
    merged_df.loc[needs_update, 'latitude'] = merged_df.loc[needs_update, 'latitude']
    merged_df.loc[needs_update, 'longitude'] = merged_df.loc[needs_update, 'longitude']

# Update the original DataFrame with the merged data
df_filtered_sorted_nieruchomosci = merged_df


df_filtered_sorted_nieruchomosci[['address', 'location', 'latitude', 'longitude']].head()

,address,location,latitude,longitude
0,"103171, Mistrzejowice, Kraków, małopolskie","Mistrzejowice, Kraków, województwo małopolskie...",50.097426,20.009603
1,"2 Pułku Lotniczego, Kraków, małopolskie","2, Osiedle 2 Pułku Lotniczego, Czyżyny, Kraków...",50.077806,20.012307
2,"2 Pułku Lotniczego, Kraków, małopolskie","2, Osiedle 2 Pułku Lotniczego, Czyżyny, Kraków...",50.077806,20.012307
3,"28 Lipca 1943, Wola Justowska, Kraków, małopol...","28 Lipca 1943, Wola Justowska, Zwierzyniec, Kr...",50.062710,19.872002
4,"28 Lipca 1943, Wola Justowska, Kraków, małopol...","28 Lipca 1943, Wola Justowska, Zwierzyniec, Kr...",50.062710,19.872002


In [229]:
df_filtered_sorted_nieruchomosci[df_filtered_sorted_nieruchomosci['location'].isna()]

,url,name/title,address,price,area,price-per-area,floor/store,no of floors/stores in the building,no of rooms,year of construction,parking space,market,form of ownership,parking space details,location,latitude,longitude


In [224]:
unique_addresses = set(df_filtered_sorted_nieruchomosci[df_filtered_sorted_nieruchomosci['location'].isna()]['address'].unique())
address_list = list(unique_addresses)

location_df_nieruchomosci = pd.DataFrame({
    'address': address_list,
    'manipulated address': address_list
})  

location_df_nieruchomosci.to_excel("temp.xlsx", index=False)
location_df_nieruchomosci

,address,manipulated address
0,"Armii ""Kraków"", Podgórze, Kraków, małopolskie","Armii ""Kraków"", Podgórze, Kraków, małopolskie"
1,"Bronowice Nowe, Kraków, małopolskie","Bronowice Nowe, Kraków, małopolskie"
2,"Lichecka, Łagiewniki, Kraków, małopolskie","Lichecka, Łagiewniki, Kraków, małopolskie"
3,"Armii Kraków, Jugowice, Kraków, małopolskie","Armii Kraków, Jugowice, Kraków, małopolskie"
4,"ul. Żeglarska 7, Zarzecze 34-326, Kraków, mało...","ul. Żeglarska 7, Zarzecze 34-326, Kraków, mało..."
5,"Armii Kraków, Swoszowice, Kraków, małopolskie","Armii Kraków, Swoszowice, Kraków, małopolskie"
6,"okolica ul. Mogilskiej, Grzegórzki, Kraków, ma...","okolica ul. Mogilskiej, Grzegórzki, Kraków, ma..."
7,"Gancarska, Stare Miasto, Kraków, małopolskie","Gancarska, Stare Miasto, Kraków, małopolskie"
8,"Kazimierza Siemianowicza, Płaszów, Kraków, mał...","Kazimierza Siemianowicza, Płaszów, Kraków, mał..."
9,"Łobzów, Kraków, małopolskie","Łobzów, Kraków, małopolskie"


In [225]:
location_df_nieruchomosci=geocode_and_save(pd.read_excel('temp.xlsx'), 'datasets/geocoded_from_xlsx.csv')
location_df_nieruchomosci

Rows needing geocoding: 10
Armii "Kraków", Jugowice, Swoszowice, Kraków, województwo małopolskie, 30-432, Polska, 50.0030589, 19.9260994
Bronowice, Kraków, województwo małopolskie, Polska, 50.0833247, 19.869783648014888
Lechicka, Łagiewniki, Łagiewniki-Borek Fałęcki, Kraków, województwo małopolskie, 30-601, Polska, 50.0241242, 19.9369609
Armii "Kraków", Jugowice, Swoszowice, Kraków, województwo małopolskie, 30-432, Polska, 50.0030589, 19.9260994
7, Żeglarska, Na Stawach, Zarzecze, gmina Łodygowice, powiat żywiecki, 34-326, Polska, 49.72273145, 19.182580935031844
Armii "Kraków", Jugowice, Swoszowice, Kraków, województwo małopolskie, 30-432, Polska, 50.0030589, 19.9260994
Mogilska, Osiedle Oficerskie, Grzegórzki, Kraków, województwo małopolskie, 31-525, Polska, 50.0662338, 19.9614209
Garncarska, Piasek, Stare Miasto, Kraków, województwo małopolskie, 31-115, Polska, 50.0626415, 19.9277117
Bagry - ul. Lipska/Siemienowicza, Płaszów, Podgórze, Kraków, województwo małopolskie, Polska, 50.0380

,address,manipulated address,location,latitude,longitude
0,"Armii ""Kraków"", Podgórze, Kraków, małopolskie","Armii Kraków, Kraków, małopolskie","Armii ""Kraków"", Jugowice, Swoszowice, Kraków, ...",50.003059,19.926099
1,"Bronowice Nowe, Kraków, małopolskie","Bronowice, Kraków, małopolskie","Bronowice, Kraków, województwo małopolskie, Po...",50.083325,19.869784
2,"Lichecka, Łagiewniki, Kraków, małopolskie","Lechicka, Łagiewniki, Kraków, małopolskie","Lechicka, Łagiewniki, Łagiewniki-Borek Fałęcki...",50.024124,19.936961
3,"Armii Kraków, Jugowice, Kraków, małopolskie","Armii Kraków, Jugowice, Kraków, małopolskie","Armii ""Kraków"", Jugowice, Swoszowice, Kraków, ...",50.003059,19.926099
4,"ul. Żeglarska 7, Zarzecze 34-326, Kraków, mało...","Żeglarska 7, Zarzecze 34-326","7, Żeglarska, Na Stawach, Zarzecze, gmina Łody...",49.722731,19.182581
5,"Armii Kraków, Swoszowice, Kraków, małopolskie","Armii Kraków, Kraków, małopolskie","Armii ""Kraków"", Jugowice, Swoszowice, Kraków, ...",50.003059,19.926099
6,"okolica ul. Mogilskiej, Grzegórzki, Kraków, ma...","Mogilska, Grzegórzki, Kraków, małopolskie","Mogilska, Osiedle Oficerskie, Grzegórzki, Krak...",50.066234,19.961421
7,"Gancarska, Stare Miasto, Kraków, małopolskie","Garncarska, Kraków, małopolskie","Garncarska, Piasek, Stare Miasto, Kraków, woje...",50.062641,19.927712
8,"Kazimierza Siemianowicza, Płaszów, Kraków, mał...","Siemienowicza, Płaszów, Kraków, małopolskie","Bagry - ul. Lipska/Siemienowicza, Płaszów, Pod...",50.038014,19.995590
9,"Łobzów, Kraków, małopolskie","Łobzów, Kraków, małopolskie","Łobzów, Krowodrza, Kraków, województwo małopol...",50.075270,19.907778


In [230]:
df_filtered_sorted_nieruchomosci['distance'] = df_filtered_sorted_nieruchomosci.apply(
    lambda row: haversine(city_center_lat, city_center_lon, row['latitude'], row['longitude']),
    axis=1
)
df_filtered_sorted_nieruchomosci

,url,name/title,address,price,area,price-per-area,floor/store,no of floors/stores in the building,no of rooms,year of construction,parking space,market,form of ownership,parking space details,location,latitude,longitude,distance
0,https://krakow.nieruchomosci-online.pl/mieszka...,Mieszkanie Kraków,"103171, Mistrzejowice, Kraków, małopolskie",1087000.0,74.80,14532.09,6,7,3,2016.0,True,wtórny,pełna własność,w garażu podziemnym,"Mistrzejowice, Kraków, województwo małopolskie...",50.097426,20.009603,6.500728
1,https://krakow.nieruchomosci-online.pl/mieszka...,Mieszkanie Kraków,"2 Pułku Lotniczego, Kraków, małopolskie",550000.0,32.00,17187.50,4,8,1,NaN,False,wtórny,NaN,parking publiczny / na ulicy,"2, Osiedle 2 Pułku Lotniczego, Czyżyny, Kraków...",50.077806,20.012307,5.639111
2,https://krakow.nieruchomosci-online.pl/mieszka...,Mieszkanie Kraków,"2 Pułku Lotniczego, Kraków, małopolskie",550000.0,32.30,17028.00,4,10,1,1985.0,False,wtórny,NaN,parking publiczny / na ulicy,"2, Osiedle 2 Pułku Lotniczego, Czyżyny, Kraków...",50.077806,20.012307,5.639111
3,https://krakow.nieruchomosci-online.pl/mieszka...,"Apartament, ul. 28 Lipca 1943","28 Lipca 1943, Wola Justowska, Kraków, małopol...",2290000.0,155.16,14758.96,0,2,5,2009.0,True,wtórny,pełna własność,garaż wolnostojący,"28 Lipca 1943, Wola Justowska, Zwierzyniec, Kr...",50.062710,19.872002,4.662523
4,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. 28 Lipca 1943","28 Lipca 1943, Wola Justowska, Kraków, małopol...",2399000.0,139.00,17258.99,1,2,3,2022.0,False,wtórny,pełna własność,parking publiczny / na ulicy,"28 Lipca 1943, Wola Justowska, Zwierzyniec, Kr...",50.062710,19.872002,4.662523
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8073,https://krakow.nieruchomosci-online.pl/mieszka...,"Kawalerka, ul. Żółkiewskiego","Żółkiewskiego, Grzegórzki, Kraków, małopolskie",532000.0,24.40,21803.28,1,2,1,1900.0,True,wtórny,NaN,tak,"Hetmana Stanisława Żółkiewskiego, Wesoła, Grze...",50.057738,19.957939,1.542326
8074,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Żółkiewskiego","Żółkiewskiego, Grzegórzki, Kraków, małopolskie",695000.0,36.97,18799.03,1,2,2,1900.0,True,wtórny,NaN,tak,"Hetmana Stanisława Żółkiewskiego, Wesoła, Grze...",50.057738,19.957939,1.542326
8075,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Żółkiewskiego","Żółkiewskiego, Grzegórzki, Kraków, małopolskie",695000.0,37.98,18299.10,0,2,1,1900.0,False,wtórny,NaN,parking strzeżony w pobliżu,"Hetmana Stanisława Żółkiewskiego, Wesoła, Grze...",50.057738,19.957939,1.542326
8076,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Żółkiewskiego","Żółkiewskiego, Grzegórzki, Kraków, małopolskie",1698500.0,79.00,21500.00,2,4,4,2024.0,False,pierwotny,pełna własność,parking publiczny / na ulicy,"Hetmana Stanisława Żółkiewskiego, Wesoła, Grze...",50.057738,19.957939,1.542326


In [231]:
# Extracting districts
df_filtered_sorted_nieruchomosci['district'] = df_filtered_sorted_nieruchomosci['location'].str.extract(r'(?:^|, )([^,]+), Kraków')
df_filtered_sorted_nieruchomosci

,url,name/title,address,price,area,price-per-area,floor/store,no of floors/stores in the building,no of rooms,year of construction,parking space,market,form of ownership,parking space details,location,latitude,longitude,distance,district
0,https://krakow.nieruchomosci-online.pl/mieszka...,Mieszkanie Kraków,"103171, Mistrzejowice, Kraków, małopolskie",1087000.0,74.80,14532.09,6,7,3,2016.0,True,wtórny,pełna własność,w garażu podziemnym,"Mistrzejowice, Kraków, województwo małopolskie...",50.097426,20.009603,6.500728,Mistrzejowice
1,https://krakow.nieruchomosci-online.pl/mieszka...,Mieszkanie Kraków,"2 Pułku Lotniczego, Kraków, małopolskie",550000.0,32.00,17187.50,4,8,1,NaN,False,wtórny,NaN,parking publiczny / na ulicy,"2, Osiedle 2 Pułku Lotniczego, Czyżyny, Kraków...",50.077806,20.012307,5.639111,Czyżyny
2,https://krakow.nieruchomosci-online.pl/mieszka...,Mieszkanie Kraków,"2 Pułku Lotniczego, Kraków, małopolskie",550000.0,32.30,17028.00,4,10,1,1985.0,False,wtórny,NaN,parking publiczny / na ulicy,"2, Osiedle 2 Pułku Lotniczego, Czyżyny, Kraków...",50.077806,20.012307,5.639111,Czyżyny
3,https://krakow.nieruchomosci-online.pl/mieszka...,"Apartament, ul. 28 Lipca 1943","28 Lipca 1943, Wola Justowska, Kraków, małopol...",2290000.0,155.16,14758.96,0,2,5,2009.0,True,wtórny,pełna własność,garaż wolnostojący,"28 Lipca 1943, Wola Justowska, Zwierzyniec, Kr...",50.062710,19.872002,4.662523,Zwierzyniec
4,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. 28 Lipca 1943","28 Lipca 1943, Wola Justowska, Kraków, małopol...",2399000.0,139.00,17258.99,1,2,3,2022.0,False,wtórny,pełna własność,parking publiczny / na ulicy,"28 Lipca 1943, Wola Justowska, Zwierzyniec, Kr...",50.062710,19.872002,4.662523,Zwierzyniec
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8073,https://krakow.nieruchomosci-online.pl/mieszka...,"Kawalerka, ul. Żółkiewskiego","Żółkiewskiego, Grzegórzki, Kraków, małopolskie",532000.0,24.40,21803.28,1,2,1,1900.0,True,wtórny,NaN,tak,"Hetmana Stanisława Żółkiewskiego, Wesoła, Grze...",50.057738,19.957939,1.542326,Grzegórzki
8074,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Żółkiewskiego","Żółkiewskiego, Grzegórzki, Kraków, małopolskie",695000.0,36.97,18799.03,1,2,2,1900.0,True,wtórny,NaN,tak,"Hetmana Stanisława Żółkiewskiego, Wesoła, Grze...",50.057738,19.957939,1.542326,Grzegórzki
8075,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Żółkiewskiego","Żółkiewskiego, Grzegórzki, Kraków, małopolskie",695000.0,37.98,18299.10,0,2,1,1900.0,False,wtórny,NaN,parking strzeżony w pobliżu,"Hetmana Stanisława Żółkiewskiego, Wesoła, Grze...",50.057738,19.957939,1.542326,Grzegórzki
8076,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Żółkiewskiego","Żółkiewskiego, Grzegórzki, Kraków, małopolskie",1698500.0,79.00,21500.00,2,4,4,2024.0,False,pierwotny,pełna własność,parking publiczny / na ulicy,"Hetmana Stanisława Żółkiewskiego, Wesoła, Grze...",50.057738,19.957939,1.542326,Grzegórzki


In [232]:
print(df_filtered_sorted_nieruchomosci[df_filtered_sorted_nieruchomosci['district'].isna()].shape[0])
df_filtered_sorted_nieruchomosci = df_filtered_sorted_nieruchomosci.dropna(subset=['district'])

39


In [ ]:
# Filling in
df_filtered_sorted_nieruchomosci['year of construction'] = df_filtered_sorted_nieruchomosci['year of construction'].fillna(new_mean_year)
df_filtered_sorted_nieruchomosci['form of ownership'] = df_filtered_sorted_nieruchomosci['form of ownership'].fillna('pełna własność')
df_filtered_sorted_nieruchomosci.to_csv('datasets/apartments_nieruchomosci.csv')

In [234]:
df_filtered_sorted_nieruchomosci.isnull().sum()

url                                    0
name/title                             0
address                                0
price                                  0
area                                   0
price-per-area                         0
floor/store                            0
no of floors/stores in the building    0
no of rooms                            0
year of construction                   0
parking space                          0
market                                 0
form of ownership                      0
parking space details                  0
location                               0
latitude                               0
longitude                              0
distance                               0
district                               0
dtype: int64

## Removing Duplicates, Geolocation, Filling NaNs, and Exporting Otodom Dataset

In [ ]:
df_filtered_sorted_otodom = remove_duplicates(df_otodom)
df_filtered_sorted_otodom.drop('is_duplicate', axis=1, inplace=True)
df_filtered_sorted_otodom

In [236]:
df_filtered_sorted_otodom['address'] = df_filtered_sorted_otodom.apply(update_address, axis=1)
df_filtered_sorted_otodom['address'] = df_filtered_sorted_otodom['address'].str.replace(r'^ul\.\s+', '', regex=True)
df_filtered_sorted_otodom['address'] = df_filtered_sorted_otodom['address'].str.replace(r',\s*ul\.\s[^,\d]+,\s', ', ', regex=True)
df_filtered_sorted_otodom

,url,name/title,address,price,area,price-per-area,floor/store,no of floors/stores in the building,no of rooms,year of construction,...,market,form of ownership,condition,rent,heating,advertiser type,elevator,outdoor area,building type,building material
4499,https://www.otodom.pl/pl/oferta/nowe-mieszkani...,Nowe mieszkania na Górce Narodowej,"29 listopada - okolice, Górka Narodowa, Prądni...",778689.0,61.3,12703.0,1,5,2,2023.0,...,pierwotny,pełna własność,NaN,NaN,miejskie,biuro nieruchomości,True,balkon,brak informacji,brak informacji
1143,https://www.otodom.pl/pl/oferta/balkon-3-pokoj...,balkon + 3 pokoje + parking,"Adama Vetulaniego, Prądnik Biały, Prądnik Biał...",970000.0,55.0,17636.0,4,7,3,2023.0,...,wtórny,pełna własność,NaN,450.0,miejskie,biuro nieruchomości,True,balkon,blok,brak informacji
8954,https://www.otodom.pl/pl/oferta/zlocien-2-poko...,"Złocień, 2 pokoje, balkon","Agatowa, Domagały,, Złocień, Bieżanów-Prokocim...",625000.0,36.8,16984.0,4,4,2,2019.0,...,wtórny,pełna własność,do zamieszkania,425.0,miejskie,biuro nieruchomości,True,balkon,blok,brak informacji
2385,https://www.otodom.pl/pl/oferta/komfortowe-2-p...,komfortowe 2-pokojowe mieszkanie 42m2,"Albertyńskie, Bieńczyce, Bieńczyce, Kraków, ma...",559000.0,42.0,13310.0,2,4,2,1964.0,...,wtórny,NaN,NaN,500.0,miejskie,biuro nieruchomości,False,balkon,blok,brak informacji
3985,https://www.otodom.pl/pl/oferta/aleja-29-listo...,Aleja 29 Listopada obok UR,"Aleja 29 Listopada, Górka Narodowa, Prądnik Bi...",695000.0,48.1,14449.0,3,3,2,NaN,...,wtórny,pełna własność,do zamieszkania,NaN,gazowe,biuro nieruchomości,False,balkon,blok,brak informacji
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,https://www.otodom.pl/pl/oferta/3-pokoje-na-za...,3 pokoje na Żabińcu !!!,"Żabiniec, Prądnik Biały, Kraków, małopolskie",850000.0,77.0,11039.0,4,6,3,2002.0,...,wtórny,pełna własność,NaN,800.0,miejskie,biuro nieruchomości,True,balkon,blok,brak informacji
7127,https://www.otodom.pl/pl/oferta/pradnik-bialy-...,Prądnik Biały - Ul. Reja,"Żabiniec, Prądnik Biały, Kraków, małopolskie",1100000.0,71.0,15493.0,0,6,3,2005.0,...,wtórny,NaN,do zamieszkania,NaN,miejskie,biuro nieruchomości,True,balkon,blok,brak informacji
7233,https://www.otodom.pl/pl/oferta/mieszkanie-73-...,"Mieszkanie, 73 m², Kraków","Żabiniec, Prądnik Biały, Kraków, małopolskie",1100000.0,73.0,15068.0,0,6,3,2005.0,...,wtórny,pełna własność,NaN,700.0,miejskie,biuro nieruchomości,True,balkon,blok,brak informacji
388,https://www.otodom.pl/pl/oferta/widokowe-m3-z-...,"Widokowe M3 z klimatyzacją i parkingiem, Żabiniec","Żabiniec, Prądnik Biały, Kraków, małopolskie",1179000.0,71.5,16490.0,5,8,3,2001.0,...,wtórny,pełna własność,do zamieszkania,940.0,miejskie,biuro nieruchomości,True,balkon,apartamentowiec,brak informacji


In [237]:
merged_df = pd.merge(
    df_filtered_sorted_otodom, 
    location_df[['address', 'location', 'latitude', 'longitude']], 
    on='address', 
    how='left'
)

needs_update = merged_df['location'].isna()

merged_df.loc[needs_update, 'location'] = merged_df.loc[needs_update, 'location']
merged_df.loc[needs_update, 'latitude'] = merged_df.loc[needs_update, 'latitude']
merged_df.loc[needs_update, 'longitude'] = merged_df.loc[needs_update, 'longitude']


df_filtered_sorted_otodom = merged_df

In [238]:
df_filtered_sorted_otodom[df_filtered_sorted_otodom['location'].isna()]

,url,name/title,address,price,area,price-per-area,floor/store,no of floors/stores in the building,no of rooms,year of construction,...,rent,heating,advertiser type,elevator,outdoor area,building type,building material,location,latitude,longitude
3919,https://www.otodom.pl/pl/oferta/mieszkanie-ul-...,Mieszkanie ul. Bajeczna - wysoki standard,"Bajeczna - wysoki standard, Dąbie, Grzegórzki,...",1189000.0,56.67,20981.0,2,5,3,2023.0,...,650.0,NaN,prywatny,True,balkon,blok,brak informacji,NaN,NaN,NaN
7187,https://www.otodom.pl/pl/oferta/29-metrow-przy...,29 metrów przy ul. Pękowickiej - balkon i komórka,"Pękowickiej - balkon i komórka, Tonie, Prądnik...",492999.0,29.00,17000.0,3,3,1,2019.0,...,310.0,miejskie,biuro nieruchomości,False,None,blok,brak informacji,NaN,NaN,NaN
9848,https://www.otodom.pl/pl/oferta/mieszkanie-ul-...,Mieszkanie ul. Lea po generalnym remoncie,"Lea po generalnym remoncie, Łobzów, Krowodrza,...",995000.0,48.00,20729.0,7,10,3,1990.0,...,NaN,miejskie,prywatny,True,balkon,blok,brak informacji,NaN,NaN,NaN


In [240]:
unique_addresses = set(df_filtered_sorted_otodom[df_filtered_sorted_otodom['location'].isna()]['address'].unique())
address_list = list(unique_addresses)

location_df_otodom = pd.DataFrame({
    'address': address_list,
    'manipulated address': address_list
})

location_df_otodom.to_excel("temp.xlsx", index=False)

In [241]:
location_df_otodom=geocode_and_save(pd.read_excel('temp.xlsx'), 'datasets/geocoded_from_xlsx.csv')
location_df_otodom

Rows needing geocoding: 3
Bajeczna, Dąbie, Grzegórzki, Kraków, województwo małopolskie, 31-566, Polska, 50.0574115, 19.9766744
Juliusza Lea, Łobzów, Krowodrza, Kraków, województwo małopolskie, 30-070, Polska, 50.0723293, 19.9050999
Pękowicka, Tonie, Prądnik Biały, Kraków, gmina Zielonki, powiat krakowski, województwo małopolskie, 31-267, Polska, 50.1195714, 19.9069499
Data saved to datasets/geocoded_from_xlsx.csv


,address,manipulated address,location,latitude,longitude
0,"Bajeczna - wysoki standard, Dąbie, Grzegórzki,...","Bajeczna, Dąbie, Grzegórzki, Kraków, małopolskie","Bajeczna, Dąbie, Grzegórzki, Kraków, województ...",50.057412,19.976674
1,"Lea po generalnym remoncie, Łobzów, Krowodrza,...","Lea , Łobzów, Krowodrza, Kraków, małopolskie","Juliusza Lea, Łobzów, Krowodrza, Kraków, wojew...",50.072329,19.905100
2,"Pękowickiej - balkon i komórka, Tonie, Prądnik...","Pękowicka, Tonie, Prądnik Biały, Kraków, małop...","Pękowicka, Tonie, Prądnik Biały, Kraków, gmina...",50.119571,19.906950


In [242]:
merged_df = pd.merge(
    df_filtered_sorted_otodom,
    location_df_otodom[['address', 'location', 'latitude', 'longitude']],
    on='address',
    how='left',
    suffixes=('', '_from_location')
)
if 'location_from_location' in merged_df.columns:
    merged_df['location'] = merged_df['location_from_location'].combine_first(merged_df['location'])
    merged_df['latitude'] = merged_df['latitude_from_location'].combine_first(merged_df['latitude'])
    merged_df['longitude'] = merged_df['longitude_from_location'].combine_first(merged_df['longitude'])
    merged_df.drop(['location_from_location', 'latitude_from_location', 'longitude_from_location'], axis=1, inplace=True)

df_filtered_sorted_otodom.update(merged_df)

In [243]:
df_filtered_sorted_otodom['distance'] = df_filtered_sorted_otodom.apply(
    lambda row: haversine(city_center_lat, city_center_lon, row['latitude'], row['longitude']),
    axis=1
)
df_filtered_sorted_otodom

,url,name/title,address,price,area,price-per-area,floor/store,no of floors/stores in the building,no of rooms,year of construction,...,heating,advertiser type,elevator,outdoor area,building type,building material,location,latitude,longitude,distance
0,https://www.otodom.pl/pl/oferta/nowe-mieszkani...,Nowe mieszkania na Górce Narodowej,"29 listopada - okolice, Górka Narodowa, Prądni...",778689.0,61.3,12703.0,1,5,2,2023.0,...,miejskie,biuro nieruchomości,True,balkon,brak informacji,brak informacji,"Aleja 29 Listopada, Górka Narodowa, Prądnik Bi...",50.103296,19.963896,4.983478
1,https://www.otodom.pl/pl/oferta/balkon-3-pokoj...,balkon + 3 pokoje + parking,"Adama Vetulaniego, Prądnik Biały, Prądnik Biał...",970000.0,55.0,17636.0,4,7,3,2023.0,...,miejskie,biuro nieruchomości,True,balkon,blok,brak informacji,"Adama Vetulaniego, Prądnik Biały, Kraków, woje...",50.095951,19.924784,3.895335
2,https://www.otodom.pl/pl/oferta/zlocien-2-poko...,"Złocień, 2 pokoje, balkon","Agatowa, Domagały,, Złocień, Bieżanów-Prokocim...",625000.0,36.8,16984.0,4,4,2,2019.0,...,miejskie,biuro nieruchomości,True,balkon,blok,brak informacji,"Agatowa, Osiedle Kolejowe, Złocień, Bieżanów-P...",50.022046,20.046131,8.943634
3,https://www.otodom.pl/pl/oferta/komfortowe-2-p...,komfortowe 2-pokojowe mieszkanie 42m2,"Albertyńskie, Bieńczyce, Bieńczyce, Kraków, ma...",559000.0,42.0,13310.0,2,4,2,1964.0,...,miejskie,biuro nieruchomości,False,balkon,blok,brak informacji,"Osiedle Albertyńskie, Bieńczyce, Kraków, wojew...",50.081431,20.017628,6.131841
4,https://www.otodom.pl/pl/oferta/aleja-29-listo...,Aleja 29 Listopada obok UR,"Aleja 29 Listopada, Górka Narodowa, Prądnik Bi...",695000.0,48.1,14449.0,3,3,2,NaN,...,gazowe,biuro nieruchomości,False,balkon,blok,brak informacji,"Aleja 29 Listopada, Górka Narodowa, Prądnik Bi...",50.099768,19.962839,4.592695
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9890,https://www.otodom.pl/pl/oferta/3-pokoje-na-za...,3 pokoje na Żabińcu !!!,"Żabiniec, Prądnik Biały, Kraków, małopolskie",850000.0,77.0,11039.0,4,6,3,2002.0,...,miejskie,biuro nieruchomości,True,balkon,blok,brak informacji,"Żabiniec, Prądnik Biały, Kraków, województwo m...",50.085931,19.941743,2.695967
9891,https://www.otodom.pl/pl/oferta/pradnik-bialy-...,Prądnik Biały - Ul. Reja,"Żabiniec, Prądnik Biały, Kraków, małopolskie",1100000.0,71.0,15493.0,0,6,3,2005.0,...,miejskie,biuro nieruchomości,True,balkon,blok,brak informacji,"Żabiniec, Prądnik Biały, Kraków, województwo m...",50.085931,19.941743,2.695967
9892,https://www.otodom.pl/pl/oferta/mieszkanie-73-...,"Mieszkanie, 73 m², Kraków","Żabiniec, Prądnik Biały, Kraków, małopolskie",1100000.0,73.0,15068.0,0,6,3,2005.0,...,miejskie,biuro nieruchomości,True,balkon,blok,brak informacji,"Żabiniec, Prądnik Biały, Kraków, województwo m...",50.085931,19.941743,2.695967
9893,https://www.otodom.pl/pl/oferta/widokowe-m3-z-...,"Widokowe M3 z klimatyzacją i parkingiem, Żabiniec","Żabiniec, Prądnik Biały, Kraków, małopolskie",1179000.0,71.5,16490.0,5,8,3,2001.0,...,miejskie,biuro nieruchomości,True,balkon,apartamentowiec,brak informacji,"Żabiniec, Prądnik Biały, Kraków, województwo m...",50.085931,19.941743,2.695967


In [244]:
# Extracting districts
df_filtered_sorted_otodom['district'] = df_filtered_sorted_otodom['location'].str.extract(r'(?:^|, )([^,]+), Kraków')
df_filtered_sorted_otodom

,url,name/title,address,price,area,price-per-area,floor/store,no of floors/stores in the building,no of rooms,year of construction,...,advertiser type,elevator,outdoor area,building type,building material,location,latitude,longitude,distance,district
0,https://www.otodom.pl/pl/oferta/nowe-mieszkani...,Nowe mieszkania na Górce Narodowej,"29 listopada - okolice, Górka Narodowa, Prądni...",778689.0,61.3,12703.0,1,5,2,2023.0,...,biuro nieruchomości,True,balkon,brak informacji,brak informacji,"Aleja 29 Listopada, Górka Narodowa, Prądnik Bi...",50.103296,19.963896,4.983478,Prądnik Biały
1,https://www.otodom.pl/pl/oferta/balkon-3-pokoj...,balkon + 3 pokoje + parking,"Adama Vetulaniego, Prądnik Biały, Prądnik Biał...",970000.0,55.0,17636.0,4,7,3,2023.0,...,biuro nieruchomości,True,balkon,blok,brak informacji,"Adama Vetulaniego, Prądnik Biały, Kraków, woje...",50.095951,19.924784,3.895335,Prądnik Biały
2,https://www.otodom.pl/pl/oferta/zlocien-2-poko...,"Złocień, 2 pokoje, balkon","Agatowa, Domagały,, Złocień, Bieżanów-Prokocim...",625000.0,36.8,16984.0,4,4,2,2019.0,...,biuro nieruchomości,True,balkon,blok,brak informacji,"Agatowa, Osiedle Kolejowe, Złocień, Bieżanów-P...",50.022046,20.046131,8.943634,Bieżanów-Prokocim
3,https://www.otodom.pl/pl/oferta/komfortowe-2-p...,komfortowe 2-pokojowe mieszkanie 42m2,"Albertyńskie, Bieńczyce, Bieńczyce, Kraków, ma...",559000.0,42.0,13310.0,2,4,2,1964.0,...,biuro nieruchomości,False,balkon,blok,brak informacji,"Osiedle Albertyńskie, Bieńczyce, Kraków, wojew...",50.081431,20.017628,6.131841,Bieńczyce
4,https://www.otodom.pl/pl/oferta/aleja-29-listo...,Aleja 29 Listopada obok UR,"Aleja 29 Listopada, Górka Narodowa, Prądnik Bi...",695000.0,48.1,14449.0,3,3,2,NaN,...,biuro nieruchomości,False,balkon,blok,brak informacji,"Aleja 29 Listopada, Górka Narodowa, Prądnik Bi...",50.099768,19.962839,4.592695,Prądnik Biały
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9890,https://www.otodom.pl/pl/oferta/3-pokoje-na-za...,3 pokoje na Żabińcu !!!,"Żabiniec, Prądnik Biały, Kraków, małopolskie",850000.0,77.0,11039.0,4,6,3,2002.0,...,biuro nieruchomości,True,balkon,blok,brak informacji,"Żabiniec, Prądnik Biały, Kraków, województwo m...",50.085931,19.941743,2.695967,Prądnik Biały
9891,https://www.otodom.pl/pl/oferta/pradnik-bialy-...,Prądnik Biały - Ul. Reja,"Żabiniec, Prądnik Biały, Kraków, małopolskie",1100000.0,71.0,15493.0,0,6,3,2005.0,...,biuro nieruchomości,True,balkon,blok,brak informacji,"Żabiniec, Prądnik Biały, Kraków, województwo m...",50.085931,19.941743,2.695967,Prądnik Biały
9892,https://www.otodom.pl/pl/oferta/mieszkanie-73-...,"Mieszkanie, 73 m², Kraków","Żabiniec, Prądnik Biały, Kraków, małopolskie",1100000.0,73.0,15068.0,0,6,3,2005.0,...,biuro nieruchomości,True,balkon,blok,brak informacji,"Żabiniec, Prądnik Biały, Kraków, województwo m...",50.085931,19.941743,2.695967,Prądnik Biały
9893,https://www.otodom.pl/pl/oferta/widokowe-m3-z-...,"Widokowe M3 z klimatyzacją i parkingiem, Żabiniec","Żabiniec, Prądnik Biały, Kraków, małopolskie",1179000.0,71.5,16490.0,5,8,3,2001.0,...,biuro nieruchomości,True,balkon,apartamentowiec,brak informacji,"Żabiniec, Prądnik Biały, Kraków, województwo m...",50.085931,19.941743,2.695967,Prądnik Biały


In [245]:
print(df_filtered_sorted_otodom[df_filtered_sorted_otodom['district'].isna()].shape[0])
df_filtered_sorted_otodom = df_filtered_sorted_otodom.dropna(subset=['district'])

3


In [ ]:
# Here begins filling in missing data
df_filtered_sorted_otodom['year of construction'] = df_filtered_sorted_otodom['year of construction'].fillna(new_mean_year)
df_filtered_sorted_otodom['rent'] = df_filtered_sorted_otodom['rent'].fillna(new_mean_rent)
df_filtered_sorted_otodom['form of ownership'] = df_filtered_sorted_otodom['form of ownership'].fillna('pełna własność')
df_filtered_sorted_otodom['condition'] = df_filtered_sorted_otodom['condition'].fillna('brak informacji')
df_filtered_sorted_otodom['heating'] = df_filtered_sorted_otodom['heating'].fillna('brak informacji')
df_filtered_sorted_otodom.to_csv('datasets/apartments_otodom.csv')
df_filtered_sorted_otodom.isnull().sum()

In [248]:
df_apartments.describe()

,price,area,price-per-area,floor/store,no of floors/stores in the building,no of rooms,year of construction
count,2.020900e+04,20209.000000,20209.000000,20209.000000,20209.000000,20209.000000,18025.000000
mean,9.587931e+05,57.364335,16799.116538,2.366124,4.820278,2.614578,2002.218696
std,6.328700e+05,28.170482,4692.962062,2.125213,2.427871,1.132654,37.467949
min,2.000000e+03,10.270000,19.140000,-1.000000,1.000000,1.000000,1300.000000
25%,6.550000e+05,40.000000,14000.000000,1.000000,3.000000,2.000000,1992.000000
50%,8.080100e+05,51.580000,16176.000000,2.000000,4.000000,3.000000,2021.000000
75%,1.045242e+06,66.630000,18628.440000,3.000000,6.000000,3.000000,2024.000000
max,1.600000e+07,550.000000,152594.090000,15.000000,18.000000,44.000000,2026.000000
